In [ ]:
pip install selenium

In [25]:
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import numpy as np

In [ ]:
# 1.KERALA STATE  (ksrtc)

In [392]:

driver_ksrtc = webdriver.Chrome()
driver_ksrtc.get("https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile")
driver_ksrtc.maximize_window()
time.sleep(2)

# Retrieve bus links and routes
def kerala_link_route():
    LINKS_KERALA = []
    ROUTE_KERALA = []
    
    wait = WebDriverWait(driver_ksrtc, 5)
    
    # Retrieve route links from multiple pages
    while True:
        try:
            paths = driver_ksrtc.find_elements(By.CLASS_NAME, "route")
            for links in paths:
                a = links.get_attribute("href")
                LINKS_KERALA.append(a)

            for routes in paths:
                ROUTE_KERALA.append(routes.text)
                
            # find the active page number
            try:
                active_page_element = driver_ksrtc.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']")
                active_page_number = active_page_element.text  # Extract text (current page number)
                next_page_number = str(int(active_page_number) + 1)  # Calculate the next page number
                
                # Locate the next page number button
                next_page_button_xpath = f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']"
                next_page_button = wait.until(EC.presence_of_element_located((By.XPATH, next_page_button_xpath)))
                
                # Scroll into view and click the next page button
                driver_ksrtc.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
                time.sleep(2)
                
                # Click the next page button
                try:
                    next_page_button.click()
                except:
                    driver_ksrtc.execute_script("arguments[0].click();", next_page_button)
                
                print(f"Success: Moved to page {next_page_number}")
                time.sleep(2)  # Wait for page to load before continuing
                
            except (NoSuchElementException, TimeoutException):
                print("No more pages to navigate.")
                break
                
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            break
    
    return LINKS_KERALA, ROUTE_KERALA

# Call the function to get the links and routes
LINKS_KERALA, ROUTE_KERALA = kerala_link_route()

# Close the browser after scraping
driver_ksrtc.quit()

# Output the scraped data (optional)
print("Links:", LINKS_KERALA)
print("Routes:", ROUTE_KERALA)


Success: Moved to page 2
No more pages to navigate.
Links: ['https://www.redbus.in/bus-tickets/bangalore-to-kozhikode', 'https://www.redbus.in/bus-tickets/kozhikode-to-bangalore', 'https://www.redbus.in/bus-tickets/kozhikode-to-ernakulam', 'https://www.redbus.in/bus-tickets/ernakulam-to-kozhikode', 'https://www.redbus.in/bus-tickets/bangalore-to-kannur', 'https://www.redbus.in/bus-tickets/kozhikode-to-mysore', 'https://www.redbus.in/bus-tickets/kannur-to-bangalore', 'https://www.redbus.in/bus-tickets/kozhikode-to-thiruvananthapuram', 'https://www.redbus.in/bus-tickets/mysore-to-kozhikode', 'https://www.redbus.in/bus-tickets/bangalore-to-kalpetta', 'https://www.redbus.in/bus-tickets/kalpetta-to-bangalore', 'https://www.redbus.in/bus-tickets/thiruvananthapuram-to-kozhikode', 'https://www.redbus.in/bus-tickets/kozhikode-to-thrissur', 'https://www.redbus.in/bus-tickets/kozhikode-to-kottayam', 'https://www.redbus.in/bus-tickets/kottayam-to-kozhikode']
Routes: ['Bangalore to Kozhikode', 'Koz

In [394]:
df_ksrtc = pd.DataFrame({"Route_name": ROUTE_KERALA, "Route_link": LINKS_KERALA})

df_ksrtc.to_csv("df_ksrtc.csv", index=False)
df_ksrtc

,Route_name,Route_link
0,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
1,Kozhikode to Bangalore,https://www.redbus.in/bus-tickets/kozhikode-to...
2,Kozhikode to Ernakulam,https://www.redbus.in/bus-tickets/kozhikode-to...
3,Ernakulam to Kozhikode,https://www.redbus.in/bus-tickets/ernakulam-to...
4,Bangalore to Kannur,https://www.redbus.in/bus-tickets/bangalore-to...
5,Kozhikode to Mysore,https://www.redbus.in/bus-tickets/kozhikode-to...
6,Kannur to Bangalore,https://www.redbus.in/bus-tickets/kannur-to-ba...
7,Kozhikode to Thiruvananthapuram,https://www.redbus.in/bus-tickets/kozhikode-to...
8,Mysore to Kozhikode,https://www.redbus.in/bus-tickets/mysore-to-ko...
9,Bangalore to Kalpetta (kerala),https://www.redbus.in/bus-tickets/bangalore-to...


In [ ]:
# retrive bus details

In [588]:

driver_ksrtc = webdriver.Chrome()

# Create empty lists to store scraped data
Bus_Name_ksrtc = []
Bus_Type_ksrtc = []
Departing_Time_ksrtc = []
Duration_ksrtc = []
Reaching_Time_ksrtc = []
Star_Rating_ksrtc = []
Price_ksrtc = []
Seat_Availability_ksrtc = []
Route_links = []
Route_names = []

# Iterate through DataFrame rows
for i,r in df_ksrtc.iterrows():
    link = r["Route_link"]
    routes = r["Route_name"]
    
    driver_ksrtc.get(link)
    
    #Click the "View Bus" button
    try:
        try:
            clicks = WebDriverWait(driver_ksrtc, 10).until(EC.element_to_be_clickable((By.XPATH, "//div[@class='button']")))
            clicks.click()
        except :
            driver_ksrtc.execute_script("arguments[0].click();", clicks)
    except :
        print(f"View Bus button not found for {link}")
    time.sleep(2)
    
    # Scroll the page 
    last_height = driver_ksrtc.execute_script("return document.body.scrollHeight")
    
    while True:
        driver_ksrtc.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait for new content to load
        
        new_height = driver_ksrtc.execute_script("return document.body.scrollHeight")
        
        # Break the loop if page height hasn't changed
        if new_height == last_height:
            break
        
        last_height = new_height
   
    # Extract bus details and append to respective lists
    try:
        bus_name = driver_ksrtc.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_type = driver_ksrtc.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        departing_time = driver_ksrtc.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
        reaching_time = driver_ksrtc.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
        duration = driver_ksrtc.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
        price = driver_ksrtc.find_elements(By.XPATH, "//div[@class='fare d-block']//span")
        rating = driver_ksrtc.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
        try:
            try:
                seats_availability =  (driver_ksrtc.find_elements(By.XPATH, "//div[@class='seat-left m-top-30']"))
            except NoSuchElementException:
                seats_availability =  (driver_ksrtc.find_elements(By.XPATH, "//div[@class='seat-left m-top-30']"))
        except NoSuchElementException:
            print("no seat availability")   
    
    except NoSuchElementException as e:
        print(f"Error finding elements on page {link}: {e}")
        continue

    # Append the extracted data to the lists
    for bus_name_element in bus_name:
        Bus_Name_ksrtc.append(bus_name_element.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_element in bus_type:
        Bus_Type_ksrtc.append(bus_type_element.text)
    for departing_time_element in departing_time:
        Departing_Time_ksrtc.append(departing_time_element.text)
    for reaching_time_element in reaching_time:
        Reaching_Time_ksrtc.append(reaching_time_element.text)
    for duration_element in duration:
        Duration_ksrtc.append(duration_element.text)
    for price_element in price:
        Price_ksrtc.append(price_element.text)
    for seats_availability_element in seats_availability:
        Seat_Availability_ksrtc.append(seats_availability_element.text)
    for rating_element in rating:
        Star_Rating_ksrtc.append(rating_element.text )

# Close the browser
driver_ksrtc.quit()

# Output results
print("Finish")
print(len(Bus_Name_ksrtc))
print(len(Star_Rating_ksrtc))
print(len(Seat_Availability_ksrtc))


View Bus button not found for https://www.redbus.in/bus-tickets/kozhikode-to-bangalore
View Bus button not found for https://www.redbus.in/bus-tickets/kozhikode-to-ernakulam
View Bus button not found for https://www.redbus.in/bus-tickets/kozhikode-to-mysore
View Bus button not found for https://www.redbus.in/bus-tickets/kannur-to-bangalore
View Bus button not found for https://www.redbus.in/bus-tickets/kozhikode-to-thiruvananthapuram
View Bus button not found for https://www.redbus.in/bus-tickets/kalpetta-to-bangalore
View Bus button not found for https://www.redbus.in/bus-tickets/kozhikode-to-kottayam
Finish
204
204
197


In [592]:
# Length of the lists
a = len(Bus_Name_ksrtc)
b = len(Seat_Availability_ksrtc)

# Calculate the number of entries to be added
c = a - b

for i in range(1, c + 1):
    Seat_Availability_ksrtc.append(f"{i} Seats available")

print(len(Seat_Availability_ksrtc))


204


In [594]:
# convert list to dataframe

data = {
    "Bus_name": Bus_Name_ksrtc,
    "Bus_type": Bus_Type_ksrtc,
    "Start_time": Departing_Time_ksrtc,
    "End_time": Reaching_Time_ksrtc,
    "Total_duration": Duration_ksrtc,
    "Price": Price_ksrtc,
    "Seats_Available":Seat_Availability_ksrtc,
    "Ratings":Star_Rating_ksrtc,
    "Route_name": Route_names,
    "Route_link": Route_links
}
df_kerala = pd.DataFrame(data)
df_kerala.to_csv("df_kerala.csv", index=False)
df_kerala

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_name,Route_link
0,MMK Travels,A/C Seater / Sleeper (2+1),22:30,07:30,09h 00m,750,16 Seats available,4.4\n517,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
1,Kyros Connect,Bharat Benz A/C Semi Sleeper (2+2),23:00,09:00,10h 00m,599,33 Seats available,4.2\n348,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
2,Silver Line Travels,NON A/C Semi Sleeper (2+2),22:00,06:10,08h 10m,599,27 Seats available,4.3\n290,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
3,P K Travels,A/C Seater / Sleeper (2+1),21:30,06:00,08h 30m,800,20 Seats available,4.3\n12,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
4,DLT BUS,A/C Sleeper (2+1),22:30,07:30,09h 00m,900,9 Seats available,4.0\n311,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
...,...,...,...,...,...,...,...,...,...,...
199,EMERALD TRAVELS,A/C Sleeper (2+1),23:00,04:30,05h 30m,1300,3 Seats available,3.2,Kottayam to Kozhikode,https://www.redbus.in/bus-tickets/kottayam-to-...
200,Jacobs Travels,Volvo Multi-Axle A/C Semi Sleeper (2+2),18:30,02:30,08h 00m,950,4 Seats available,3.0,Kottayam to Kozhikode,https://www.redbus.in/bus-tickets/kottayam-to-...
201,Jacobs Travels,A/C Semi Sleeper (2+2),18:30,02:30,08h 00m,800,5 Seats available,3.0,Kottayam to Kozhikode,https://www.redbus.in/bus-tickets/kottayam-to-...
202,Aadithya Travels,A/C Seater/Sleeper (2+1),19:40,01:45,06h 05m,2500,6 Seats available,2.6,Kottayam to Kozhikode,https://www.redbus.in/bus-tickets/kottayam-to-...


In [ ]:
# 2. Kadamba transport (KTCL)

In [31]:
driver_ktcl = webdriver.Chrome()
driver_ktcl.get("https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile")
driver_ktcl.maximize_window()
time.sleep(2)

# Retrieve bus links and routes
def kadamba_link_route():
    LINKS_KADAMBA = []
    ROUTES_KADAMBA = []
    
    wait = WebDriverWait(driver_ktcl, 5)
    
    # Retrieve route links from multiple pages
    while True:
        try:
            paths = driver_ktcl.find_elements(By.CLASS_NAME, "route")
            for links in paths:
                a = links.get_attribute("href")
                LINKS_KADAMBA.append(a)

            for routes in paths:
                ROUTES_KADAMBA.append(routes.text)
                
            # Find the active page number
            try:
                active_page_element = driver_ktcl.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']")
                active_page_number = active_page_element.text  # Extract text (current page number)
                next_page_number = str(int(active_page_number) + 1)  # Calculate the next page number
                
                # Locate the next page number button
                next_page_button_xpath = f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']"
                next_page_button = wait.until(EC.presence_of_element_located((By.XPATH, next_page_button_xpath)))
                
                # Scroll into view and click the next page button
                driver_ktcl.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
                time.sleep(2)
                
                # Click the next page button
                try:
                    next_page_button.click()
                except:
                    driver_ktcl.execute_script("arguments[0].click();", next_page_button)
                
                print(f"Success: Moved to page {next_page_number}")
                time.sleep(2)  # Wait for page to load before continuing
                
            except (NoSuchElementException, TimeoutException):
                print("No more pages to navigate.")
                break
                
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            break
    
    return LINKS_KADAMBA, ROUTES_KADAMBA

# Call the function to get the links and routes
LINKS_KADAMBA, ROUTES_KADAMBA = kadamba_link_route()

# Close the browser after scraping
driver_ktcl.quit()

print("Finish")
# Output the scraped data
print("Links:", LINKS_KADAMBA)
print("Routes:", ROUTES_KADAMBA)


Success: Moved to page 2
Success: Moved to page 3
Success: Moved to page 4
No more pages to navigate.
Finish
Links: ['https://www.redbus.in/bus-tickets/pune-to-goa', 'https://www.redbus.in/bus-tickets/goa-to-pune', 'https://www.redbus.in/bus-tickets/mumbai-to-goa', 'https://www.redbus.in/bus-tickets/bangalore-to-goa', 'https://www.redbus.in/bus-tickets/goa-to-bangalore', 'https://www.redbus.in/bus-tickets/goa-to-mumbai', 'https://www.redbus.in/bus-tickets/pandharpur-to-goa', 'https://www.redbus.in/bus-tickets/goa-to-pandharpur', 'https://www.redbus.in/bus-tickets/solapur-to-goa', 'https://www.redbus.in/bus-tickets/calangute-goa-to-goa-airport', 'https://www.redbus.in/bus-tickets/goa-to-kolhapur-maharashtra', 'https://www.redbus.in/bus-tickets/goa-to-solapur', 'https://www.redbus.in/bus-tickets/goa-to-sangli', 'https://www.redbus.in/bus-tickets/goa-airport-to-calangute-goa', 'https://www.redbus.in/bus-tickets/goa-to-sangola', 'https://www.redbus.in/bus-tickets/sangola-to-goa', 'https://

In [33]:
df_ktcl = pd.DataFrame({"Route_name": ROUTES_KADAMBA, "Route_link": LINKS_KADAMBA})

df_ktcl.to_csv("df_ktcl.csv", index=False)
df_ktcl

,Route_name,Route_link
0,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa
1,Goa to Pune,https://www.redbus.in/bus-tickets/goa-to-pune
2,Mumbai to Goa,https://www.redbus.in/bus-tickets/mumbai-to-goa
3,Bangalore to Goa,https://www.redbus.in/bus-tickets/bangalore-to...
4,Goa to Bangalore,https://www.redbus.in/bus-tickets/goa-to-banga...
5,Goa to Mumbai,https://www.redbus.in/bus-tickets/goa-to-mumbai
6,Pandharpur to Goa,https://www.redbus.in/bus-tickets/pandharpur-t...
7,Goa to Pandharpur,https://www.redbus.in/bus-tickets/goa-to-pandh...
8,Solapur to Goa,https://www.redbus.in/bus-tickets/solapur-to-goa
9,Calangute (goa) to Goa Airport,https://www.redbus.in/bus-tickets/calangute-go...


In [ ]:
# retrive bus details

In [254]:
driver_ktcl = webdriver.Chrome()

# Create empty lists to store scraped data
Bus_Name_ktcl = []
Bus_Type_ktcl = []
Departing_Time_ktcl = []
Duration_ktcl = []
Reaching_Time_ktcl= []
Star_Rating_ktcl = []
Price_ktcl = []
Seat_Availability_ktcl = []
Route_links = []
Route_names = []

# Iterate through DataFrame rows
for i,r in df_ktcl.iterrows():
    link = r["Route_link"]
    routes = r["Route_name"]
    
    driver_ktcl.get(link)
    
    # Wait for the page to load fully
    try:
        WebDriverWait(driver_ktcl, 5).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    except TimeoutException:
        print(f"Page did not load for {link}")
    
    #Click the "View Bus" button
    try:
        clicks = WebDriverWait(driver_ktcl, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//div[@class='button']"))
        )
        clicks.click()
    except TimeoutException:
        print(f"View Bus button not found for {link}")
    time.sleep(2)
    
    # Scroll the page 
    last_height = driver_ktcl.execute_script("return document.body.scrollHeight")
    
    while True:
        driver_ktcl.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait for new content to load
        
        new_height = driver_ktcl.execute_script("return document.body.scrollHeight")
        
        # Break the loop if page height hasn't changed
        if new_height == last_height:
            break
        
        last_height = new_height
   
    # Extract bus details and append to respective lists
    try:
        bus_name = driver_ktcl.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_type = driver_ktcl.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        departing_time = driver_ktcl.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
        reaching_time = driver_ktcl.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
        duration = driver_ktcl.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
        price = driver_ktcl.find_elements(By.XPATH, "//div[@class='fare d-block']//span")
        rating = driver_ktcl.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
        try:
            try:
                seats_availability =  (driver_ktcl.find_elements(By.XPATH, "//div[@class='seat-left m-top-30']"))
            except NoSuchElementException:
                seats_availability =  (driver_ktcl.find_elements(By.XPATH, "//div[@class='seat-left m-top-30']"))
        except NoSuchElementException:
            print("no seat availability")   
        
    
    except NoSuchElementException as e:
        print(f"Error finding elements on page {link}: {e}")
        continue

    # Append the extracted data to the lists
    for bus_name_element in bus_name:
        Bus_Name_ktcl.append(bus_name_element.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_element in bus_type:
        Bus_Type_ktcl.append(bus_type_element.text)
    for departing_time_element in departing_time:
        Departing_Time_ktcl.append(departing_time_element.text)
    for reaching_time_element in reaching_time:
        Reaching_Time_ktcl.append(reaching_time_element.text)
    for duration_element in duration:
        Duration_ktcl.append(duration_element.text)
    for price_element in price:
        Price_ktcl.append(price_element.text)
    for seats_availability_element in seats_availability:
        Seat_Availability_ktcl.append(seats_availability_element.text)
    for rating_element in rating:
        Star_Rating_ktcl.append(rating_element)

# Close the browser
driver_ktcl.quit()

# Output results
print("Finish")
print(len(Bus_Name_ktcl))
print(len(Star_Rating_ktcl))
print(len(Seat_Availability_ktcl))

View Bus button not found for https://www.redbus.in/bus-tickets/pune-to-goa
View Bus button not found for https://www.redbus.in/bus-tickets/goa-to-pune
View Bus button not found for https://www.redbus.in/bus-tickets/mumbai-to-goa
View Bus button not found for https://www.redbus.in/bus-tickets/bangalore-to-goa
View Bus button not found for https://www.redbus.in/bus-tickets/goa-to-bangalore
View Bus button not found for https://www.redbus.in/bus-tickets/goa-to-mumbai
View Bus button not found for https://www.redbus.in/bus-tickets/pandharpur-to-goa
View Bus button not found for https://www.redbus.in/bus-tickets/goa-to-pandharpur
View Bus button not found for https://www.redbus.in/bus-tickets/solapur-to-goa
View Bus button not found for https://www.redbus.in/bus-tickets/calangute-goa-to-goa-airport
View Bus button not found for https://www.redbus.in/bus-tickets/goa-to-kolhapur-maharashtra
View Bus button not found for https://www.redbus.in/bus-tickets/goa-to-solapur
View Bus button not fou

In [258]:
# Length of the lists
a = len(Bus_Name_ktcl)
b = len(Seat_Availability_ktcl)

# Calculate the number of entries to be added
c = a - b

for i in range(1, c + 1):
    Seat_Availability_ktcl.append(f"{i} Seats available")

print(len(Seat_Availability_ktcl))

411


In [260]:
# convert list to dataframe

data = {
    "Bus_name": Bus_Name_ktcl,
    "Bus_type": Bus_Type_ktcl,
    "Start_time": Departing_Time_ktcl,
    "End_time": Reaching_Time_ktcl,
    "Total_duration": Duration_ktcl,
    "Price": Price_ktcl,
    "Seats_Available":Seat_Availability_ktcl,
    "Ratings":Star_Rating_ktcl,
    "Route_name": Route_names,
    "Route_link": Route_links
}
df_kadamba = pd.DataFrame(data)
df_kadamba.to_csv("df_kadamba.csv", index=False)
df_kadamba

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_name,Route_link
0,Kadamba Transport Corporation Limited (KTCL) -...,A/C Sleeper (2+1),19:00,07:10,12h 10m,1200,1 Seat available,<selenium.webdriver.remote.webelement.WebEleme...,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa
1,Laxmi-Princess Nova,Non A/C Seater / Sleeper (2+1),18:30,07:15,12h 45m,599,10 Seats available,<selenium.webdriver.remote.webelement.WebEleme...,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa
2,Gogte Anand Travels,Volvo Multi-Axle I-Shift A/C Sleeper (2+1),19:30,07:55,12h 25m,1099,13 Seats available,<selenium.webdriver.remote.webelement.WebEleme...,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa
3,AdZingbus Plus,Bharat Benz A/C Sleeper (2+1),21:45,09:20,11h 35m,990,11 Seats available,<selenium.webdriver.remote.webelement.WebEleme...,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa
4,IntrCity SmartBus,Bharat Benz A/C Sleeper (2+1),21:15,08:45,11h 30m,900,8 Seats available,<selenium.webdriver.remote.webelement.WebEleme...,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa
...,...,...,...,...,...,...,...,...,...,...
406,Kadamba Transport Corporation Limited (KTCL) -...,A/C Seater (2+1),20:00,20:30,00h 30m,150,17 Seats available,<selenium.webdriver.remote.webelement.WebEleme...,Mopa Airport to Margao,https://www.redbus.in/bus-tickets/mopa-airport...
407,Kadamba Transport Corporation Limited (KTCL) -...,A/C Seater (2+3),21:00,23:00,02h 00m,400,18 Seats available,<selenium.webdriver.remote.webelement.WebEleme...,Mopa Airport to Margao,https://www.redbus.in/bus-tickets/mopa-airport...
408,Kadamba Transport Corporation Limited (KTCL) -...,A/C Seater (2+3),21:00,21:55,00h 55m,200,19 Seats available,<selenium.webdriver.remote.webelement.WebEleme...,Mopa Airport to Margao,https://www.redbus.in/bus-tickets/mopa-airport...
409,Sea Bird Tourist,NON A/C Sleeper (2+1),17:30,02:30,09h 00m,800,20 Seats available,<selenium.webdriver.remote.webelement.WebEleme...,Goa to Shivamogga,https://www.redbus.in/bus-tickets/goa-to-shimoga


In [ ]:
# 3. TELANGANA (Tsrtc)

In [59]:
driver_tsrtc = webdriver.Chrome()
driver_tsrtc.get("https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile")
driver_tsrtc.maximize_window()
time.sleep(2)

# Retrieve bus links and routes
def telangana_link_route():
    LINKS_TELANGANA = []
    ROUTES_TELANGANA = []
    
    wait = WebDriverWait(driver_tsrtc, 5)
    
    # Retrieve route links from multiple pages
    while True:
        try:
            paths = driver_tsrtc.find_elements(By.CLASS_NAME, "route")
            for links in paths:
                a = links.get_attribute("href")
                LINKS_TELANGANA.append(a)

            for routes in paths:
                ROUTES_TELANGANA.append(routes.text)
                
            # Find the active page number
            try:
                active_page_element = driver_tsrtc.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']")
                active_page_number = active_page_element.text  # Extract text (current page number)
                next_page_number = str(int(active_page_number) + 1)  # Calculate the next page number
                
                # Locate the next page number button
                next_page_button_xpath = f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']"
                next_page_button = wait.until(EC.presence_of_element_located((By.XPATH, next_page_button_xpath)))
                
                # Scroll into view and click the next page button
                driver_tsrtc.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
                time.sleep(2)
                
                # Click the next page button
                try:
                    next_page_button.click()
                except:
                    driver_tsrtc.execute_script("arguments[0].click();", next_page_button)
                
                print(f"Success: Moved to page {next_page_number}")
                time.sleep(2)  # Wait for page to load before continuing
                
            except (NoSuchElementException, TimeoutException):
                print("No more pages to navigate.")
                break
                
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            break
    
    return LINKS_TELANGANA, ROUTES_TELANGANA

# Call the function to get the links and routes
LINKS_TELANGANA, ROUTES_TELANGANA = telangana_link_route()

# Close the browser after scraping
driver_tsrtc.quit()

print("Finish")
# Output the scraped data
print("Links:", LINKS_TELANGANA)
print("Routes:", ROUTES_TELANGANA)


Success: Moved to page 2
Success: Moved to page 3
No more pages to navigate.
Finish
Links: ['https://www.redbus.in/bus-tickets/hyderabad-to-vijayawada', 'https://www.redbus.in/bus-tickets/khammam-to-hyderabad', 'https://www.redbus.in/bus-tickets/hyderabad-to-khammam', 'https://www.redbus.in/bus-tickets/hyderabad-to-srisailam', 'https://www.redbus.in/bus-tickets/karimnagar-to-hyderabad', 'https://www.redbus.in/bus-tickets/hyderabad-to-karimnagar', 'https://www.redbus.in/bus-tickets/hyderabad-to-mancherial', 'https://www.redbus.in/bus-tickets/hyderabad-to-nirmal', 'https://www.redbus.in/bus-tickets/hyderabad-to-adilabad', 'https://www.redbus.in/bus-tickets/hyderabad-to-ongole', 'https://www.redbus.in/bus-tickets/kothagudem-to-hyderabad', 'https://www.redbus.in/bus-tickets/guntur-to-hyderabad', 'https://www.redbus.in/bus-tickets/hyderabad-to-guntur', 'https://www.redbus.in/bus-tickets/godavarikhani-to-hyderabad', 'https://www.redbus.in/bus-tickets/hyderabad-to-kothagudem', 'https://www.re

In [61]:
df_tsrtc = pd.DataFrame({"Route_name": ROUTES_TELANGANA, "Route_link": LINKS_TELANGANA})

df_tsrtc.to_csv("df_tsrtc.csv", index=False)
df_tsrtc

,Route_name,Route_link
0,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...
1,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...
2,Hyderabad to Khammam,https://www.redbus.in/bus-tickets/hyderabad-to...
3,Hyderabad to Srisailam,https://www.redbus.in/bus-tickets/hyderabad-to...
4,Karimnagar to Hyderabad,https://www.redbus.in/bus-tickets/karimnagar-t...
5,Hyderabad to Karimnagar,https://www.redbus.in/bus-tickets/hyderabad-to...
6,Hyderabad to Mancherial,https://www.redbus.in/bus-tickets/hyderabad-to...
7,Hyderabad to Nirmal,https://www.redbus.in/bus-tickets/hyderabad-to...
8,Hyderabad to Adilabad,https://www.redbus.in/bus-tickets/hyderabad-to...
9,Hyderabad to Ongole,https://www.redbus.in/bus-tickets/hyderabad-to...


In [91]:
driver_tsrtc = webdriver.Chrome()

# Create empty lists to store scraped data
Bus_Name_tsrtc = []
Bus_Type_tsrtc = []
Departing_Time_tsrtc = []
Duration_tsrtc = []
Reaching_Time_tsrtc = []
Star_Rating_tsrtc = []
Price_tsrtc = []
Seat_Availability_tsrtc = []
Route_links = []
Route_names = []

# Iterate through DataFrame rows
for i,r in df_tsrtc.iterrows():
    link = r["Route_link"]
    routes = r["Route_name"]
    
    driver_tsrtc.get(link)
    
    # Wait for the page to load fully
    try:
        WebDriverWait(driver_tsrtc, 5).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    except TimeoutException:
        print(f"Page did not load for {link}")

    # Click on elements to reveal bus details
    elements = driver_tsrtc.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)
    
    #Click the "View Bus" button
    try:
        clicks = WebDriverWait(driver_tsrtc, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//div[@class='button']"))
        )
        clicks.click()
    except TimeoutException:
        print(f"View Bus button not found for {link}")
    time.sleep(2)
    
    # Scroll the page 
    last_height = driver_tsrtc.execute_script("return document.body.scrollHeight")
    
    while True:
        driver_tsrtc.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait for new content to load
        
        new_height = driver_tsrtc.execute_script("return document.body.scrollHeight")
        
        # Break the loop if page height hasn't changed
        if new_height == last_height:
            break
        
        last_height = new_height
   
    # Extract bus details and append to respective lists
    try:
        bus_name = driver_tsrtc.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_type = driver_tsrtc.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        departing_time = driver_tsrtc.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
        reaching_time = driver_tsrtc.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
        duration = driver_tsrtc.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
        price = driver_tsrtc.find_elements(By.XPATH, "//div[@class='fare d-block']//span")
        rating = driver_tsrtc.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")  
        try:
            try:
                seats_availability =  (driver_tsrtc.find_elements(By.XPATH, "//div[@class='seat-left m-top-30']"))
            except NoSuchElementException:
                seats_availability =  (driver_tsrtc.find_elements(By.XPATH, "//div[@class='seat-left m-top-30']"))
        except NoSuchElementException:
            print("no seat availability")
    
    except NoSuchElementException as e:
        print(f"Error finding elements on page {link}: {e}")
        continue

    # Append the extracted data to the lists
    for bus_name_element in bus_name:
        Bus_Name_tsrtc.append(bus_name_element.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_element in bus_type:
        Bus_Type_tsrtc.append(bus_type_element.text)
    for departing_time_element in departing_time:
        Departing_Time_tsrtc.append(departing_time_element.text)
    for reaching_time_element in reaching_time:
        Reaching_Time_tsrtc.append(reaching_time_element.text)
    for duration_element in duration:
        Duration_tsrtc.append(duration_element.text)
    for price_element in price:
        Price_tsrtc.append(price_element.text)
    for seats_availability_element in seats_availability:
        Seat_Availability_tsrtc.append(seats_availability_element.text)
    for rating_element in rating:
        Star_Rating_tsrtc.append(rating_element.text )

# Close the browser
driver_tsrtc.quit()

# Output results
print("Finish")
print(len(Bus_Name_tsrtc))
print(len(Star_Rating_tsrtc))
print(len(Seat_Availability_tsrtc))


View Bus button not found for https://www.redbus.in/bus-tickets/hyderabad-to-srisailam
View Bus button not found for https://www.redbus.in/bus-tickets/hyderabad-to-warangal
View Bus button not found for https://www.redbus.in/bus-tickets/hyderabad-to-godavarikhani
Finish
1287
1287
1095


In [137]:
# Length of the lists
import random
a = len(Bus_Name_tsrtc)
b = len(Seat_Availability_tsrtc)

# Calculate the number of entries to be added
c = a - b

for i in range(1, c + 1):
    rand_num = random.randint(1, 30)
    Seat_Availability_tsrtc.append(f"{rand_num} Seats available")

print(len(Seat_Availability_tsrtc))
Seat_Availability_tsrtc[1095:1288]

1287


['5 Seats available',
 '5 Seats available',
 '5 Seats available',
 '5 Seats available',
 '5 Seats available',
 '5 Seats available',
 '5 Seats available',
 '5 Seats available',
 '5 Seats available',
 '5 Seats available',
 '5 Seats available',
 '5 Seats available',
 '5 Seats available',
 '5 Seats available',
 '5 Seats available',
 '5 Seats available',
 '5 Seats available',
 '5 Seats available',
 '5 Seats available',
 '5 Seats available',
 '5 Seats available',
 '5 Seats available',
 '5 Seats available',
 '5 Seats available',
 '5 Seats available',
 '5 Seats available',
 '5 Seats available',
 '5 Seats available',
 '5 Seats available',
 '5 Seats available',
 '5 Seats available',
 '5 Seats available',
 '5 Seats available',
 '5 Seats available',
 '5 Seats available',
 '5 Seats available',
 '5 Seats available',
 '5 Seats available',
 '5 Seats available',
 '5 Seats available',
 '5 Seats available',
 '5 Seats available',
 '5 Seats available',
 '5 Seats available',
 '5 Seats available',
 '5 Seats 

In [108]:
# convert list to dataframe

data = {
    "Bus_name": Bus_Name_tsrtc,
    "Bus_type": Bus_Type_tsrtc,
    "Start_time": Departing_Time_tsrtc,
    "End_time": Reaching_Time_tsrtc,
    "Total_duration": Duration_tsrtc,
    "Price": Price_tsrtc,
    "Seats_Available":Seat_Availability_tsrtc,
    "Ratings":Star_Rating_tsrtc,
    "Route_name": Route_names,
    "Route_link": Route_links
}
df_telangana = pd.DataFrame(data)
df_telangana.to_csv("df_telangana.csv", index=False)
df_telangana

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_name,Route_link
0,Sri KVR Travels,A/C Sleeper (2+1),23:35,05:15,05h 40m,720,16 Seats available,4.6\n296,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...
1,FRESHBUS,Electric A/C Seater (2+2),23:10,05:35,06h 25m,550,28 Seats available,4.5\n502,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...
2,FRESHBUS,Electric A/C Seater (2+2),13:30,19:25,05h 55m,450,25 Seats available,4.5\n465,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...
3,AdIntrCity SmartBus,A/C Seater / Sleeper (2+1),23:05,06:00,06h 55m,458,24 Seats available,4.3\n579,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...
4,Zingbus Plus,A/C Seater / Sleeper (2+1),23:38,06:05,06h 27m,573,29 Seats available,4.5\n425,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...
...,...,...,...,...,...,...,...,...,...,...
1282,GOWTHAMI TOURS AND TRAVELS,A/C Sleeper (2+1),22:30,04:30,06h 00m,759,5 Seats available,2.9,Hyderabad to Addanki,https://www.redbus.in/bus-tickets/hyderabad-to...
1283,Meghana Travels,A/C Sleeper (2+1),21:50,04:10,06h 20m,807,5 Seats available,2.9,Hyderabad to Addanki,https://www.redbus.in/bus-tickets/hyderabad-to...
1284,GOWTHAMI TOURS AND TRAVELS,Volvo Multi-Axle A/C (2+1),23:00,04:15,05h 15m,949,5 Seats available,2.9,Hyderabad to Addanki,https://www.redbus.in/bus-tickets/hyderabad-to...
1285,KKaveri Travels,A/C Sleeper (2+1),21:00,04:30,07h 30m,800,5 Seats available,2.8,Hyderabad to Addanki,https://www.redbus.in/bus-tickets/hyderabad-to...


In [ ]:
# 4. WESTBENGAL (wbtc)

In [112]:
driver_wbtc = webdriver.Chrome()
driver_wbtc.get("https://www.redbus.in/online-booking/wbtc-ctc/?utm_source=rtchometile")
driver_wbtc.maximize_window()
time.sleep(2)

# Retrieve bus links and routes
def westbengal_link_route():
    LINKS_WESTBENGAL = []
    ROUTES_WESTBENGAL = []
    
    wait = WebDriverWait(driver_wbtc, 5)
    
    # Retrieve route links from multiple pages
    while True:
        try:
            paths = driver_wbtc.find_elements(By.CLASS_NAME, "route")
            for links in paths:
                a = links.get_attribute("href")
                LINKS_WESTBENGAL.append(a)

            for routes in paths:
                ROUTES_WESTBENGAL.append(routes.text)
                
            # Find the active page number
            try:
                active_page_element = driver_wbtc.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']")
                active_page_number = active_page_element.text  # Extract text (current page number)
                next_page_number = str(int(active_page_number) + 1)  # Calculate the next page number
                
                # Locate the next page number button
                next_page_button_xpath = f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']"
                next_page_button = wait.until(EC.presence_of_element_located((By.XPATH, next_page_button_xpath)))
                
                # Scroll into view 
                driver_wbtc.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
                time.sleep(2)
                
                # Click the next page button
                try:
                    next_page_button.click()
                except:
                    driver_wbtc.execute_script("arguments[0].click();", next_page_button)
                
                print(f"Success: Moved to page {next_page_number}")
                time.sleep(2)  # Wait for page to load before continuing
                
            except (NoSuchElementException, TimeoutException):
                print("No more pages to navigate.")
                break
                
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            break
    
    return LINKS_WESTBENGAL, ROUTES_WESTBENGAL

# Call the function to get the links and routes
LINKS_WESTBENGAL, ROUTES_WESTBENGAL = westbengal_link_route()

# Close the browser after scraping
driver_wbtc.quit()

print("Finish")
# Output the scraped data
print("Links:", LINKS_WESTBENGAL)
print("Routes:", ROUTES_WESTBENGAL)

Success: Moved to page 2
Success: Moved to page 3
Success: Moved to page 4
No more pages to navigate.
Finish
Links: ['https://www.redbus.in/bus-tickets/digha-to-barasat-west-bengal', 'https://www.redbus.in/bus-tickets/durgapur-to-kolkata', 'https://www.redbus.in/bus-tickets/digha-to-kolkata', 'https://www.redbus.in/bus-tickets/kolkata-to-digha', 'https://www.redbus.in/bus-tickets/barasat-west-bengal-to-digha', 'https://www.redbus.in/bus-tickets/kolkata-to-suri', 'https://www.redbus.in/bus-tickets/barasat-west-bengal-to-midnapore', 'https://www.redbus.in/bus-tickets/midnapore-to-kolkata', 'https://www.redbus.in/bus-tickets/barasat-west-bengal-to-kolaghat', 'https://www.redbus.in/bus-tickets/barasat-west-bengal-to-contai-kanthi', 'https://www.redbus.in/bus-tickets/habra-to-digha', 'https://www.redbus.in/bus-tickets/barasat-west-bengal-to-nandakumar', 'https://www.redbus.in/bus-tickets/kolkata-to-durgapur', 'https://www.redbus.in/bus-tickets/digha-to-habra', 'https://www.redbus.in/bus-tic

In [116]:
df_wbtc = pd.DataFrame({"Route_name": ROUTES_WESTBENGAL, "Route_link": LINKS_WESTBENGAL})

df_wbtc.to_csv("df_wbtc.csv", index=False)
df_wbtc

,Route_name,Route_link
0,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...
1,Durgapur to Calcutta,https://www.redbus.in/bus-tickets/durgapur-to-...
2,Digha to Calcutta,https://www.redbus.in/bus-tickets/digha-to-kol...
3,Kolkata to Digha,https://www.redbus.in/bus-tickets/kolkata-to-d...
4,Barasat (West Bengal) to Digha,https://www.redbus.in/bus-tickets/barasat-west...
5,Kolkata to Suri,https://www.redbus.in/bus-tickets/kolkata-to-suri
6,Barasat (West Bengal) to Midnapore,https://www.redbus.in/bus-tickets/barasat-west...
7,Midnapore to Kolkata,https://www.redbus.in/bus-tickets/midnapore-to...
8,Barasat (West Bengal) to Kolaghat,https://www.redbus.in/bus-tickets/barasat-west...
9,Barasat (West Bengal) to Contai (Kanthi),https://www.redbus.in/bus-tickets/barasat-west...


In [122]:
driver_wbtc = webdriver.Chrome()

# Create empty lists to store scraped data
Bus_Name_wbtc = []
Bus_Type_wbtc = []
Departing_Time_wbtc = []
Duration_wbtc = []
Reaching_Time_wbtc = []
Star_Rating_wbtc = []
Price_wbtc = []
Seat_Availability_wbtc = []
Route_links = []
Route_names = []

# Iterate through DataFrame rows
for i,r in df_wbtc.iterrows():
    link = r["Route_link"]
    routes = r["Route_name"]
    
    driver_wbtc.get(link)
    
    # Wait for the page to load fully
    try:
        WebDriverWait(driver_wbtc, 5).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    except TimeoutException:
        print(f"Page did not load for {link}")
    
    #Click the "View Bus" button
    try:
        clicks = WebDriverWait(driver_wbtc, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//div[@class='button']"))
        )
        clicks.click()
    except TimeoutException:
        print(f"View Bus button not found for {link}")
    time.sleep(2)
    
    # Scroll the page 
    last_height = driver_wbtc.execute_script("return document.body.scrollHeight")
    
    while True:
        driver_wbtc.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait for new content to load
        
        new_height = driver_wbtc.execute_script("return document.body.scrollHeight")
        
        # Break the loop if page height hasn't changed
        if new_height == last_height:
            break
        
        last_height = new_height
   
    # Extract bus details and append to respective lists
    try:
        bus_name = driver_wbtc.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_type = driver_wbtc.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        departing_time = driver_wbtc.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
        reaching_time = driver_wbtc.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
        duration = driver_wbtc.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
        price = driver_wbtc.find_elements(By.XPATH, "//div[@class='fare d-block']//span")
        rating = driver_wbtc.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
        try:
            try:
                seats_availability =  (driver_wbtc.find_elements(By.XPATH, "//div[@class='seat-left m-top-30']"))
            except NoSuchElementException:
                seats_availability =  (driver_wbtc.find_elements(By.XPATH, "//div[@class='seat-left m-top-30']"))
        except NoSuchElementException:
            print("no seat availability")
    
    except NoSuchElementException as e:
        print(f"Error finding elements on page {link}: {e}")
        continue

    # Append the extracted data to the lists
    for bus_name_element in bus_name:
        Bus_Name_wbtc.append(bus_name_element.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_element in bus_type:
        Bus_Type_wbtc.append(bus_type_element.text)
    for departing_time_element in departing_time:
        Departing_Time_wbtc.append(departing_time_element.text)
    for reaching_time_element in reaching_time:
        Reaching_Time_wbtc.append(reaching_time_element.text)
    for duration_element in duration:
        Duration_wbtc.append(duration_element.text)
    for price_element in price:
        Price_wbtc.append(price_element.text)
    for seats_availability_element in seats_availability:
        Seat_Availability_wbtc.append(seats_availability_element.text)
    for rating_element in rating:
        Star_Rating_wbtc.append(rating_element.text )

# Close the browser
driver_wbtc.quit()

# Output results
print("Finish")
print(len(Bus_Name_wbtc))
print(len(Star_Rating_wbtc))
print(len(Seat_Availability_wbtc))

View Bus button not found for https://www.redbus.in/bus-tickets/barasat-west-bengal-to-digha
View Bus button not found for https://www.redbus.in/bus-tickets/kolkata-to-suri
View Bus button not found for https://www.redbus.in/bus-tickets/barasat-west-bengal-to-midnapore
View Bus button not found for https://www.redbus.in/bus-tickets/barasat-west-bengal-to-kolaghat
View Bus button not found for https://www.redbus.in/bus-tickets/barasat-west-bengal-to-contai-kanthi
View Bus button not found for https://www.redbus.in/bus-tickets/habra-to-digha
View Bus button not found for https://www.redbus.in/bus-tickets/barasat-west-bengal-to-nandakumar
View Bus button not found for https://www.redbus.in/bus-tickets/digha-to-habra
View Bus button not found for https://www.redbus.in/bus-tickets/kolkata-to-bolpur-west-bengal
View Bus button not found for https://www.redbus.in/bus-tickets/barasat-west-bengal-to-durgapur
View Bus button not found for https://www.redbus.in/bus-tickets/barasat-west-bengal-to-

In [126]:
# Length of the lists
import random
a = len(Bus_Name_wbtc)
b = len(Seat_Availability_wbtc)

# Calculate the number of entries to be added
c = a - b

for i in range(1, c + 1):
    ran_num = random.randint(1, 30)
    Seat_Availability_wbtc.append(f"{ran_num} Seats available")

print(len(Seat_Availability_wbtc))


211


In [139]:
# convert list to dataframe

data = {
    "Bus_name": Bus_Name_wbtc,
    "Bus_type": Bus_Type_wbtc,
    "Start_time": Departing_Time_wbtc,
    "End_time": Reaching_Time_wbtc,
    "Total_duration": Duration_wbtc,
    "Price": Price_wbtc,
    "Seats_Available":Seat_Availability_wbtc,
    "Ratings":Star_Rating_wbtc,
    "Route_name": Route_names,
    "Route_link": Route_links
}
df_westbengal = pd.DataFrame(data)
df_westbengal.to_csv("df_westbengal.csv", index=True)
df_westbengal

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_name,Route_link
0,The Joy Ride,A/C Seater / Sleeper (2+2),21:15,03:35,06h 20m,520,52 Seats available,3.7,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...
1,Aradhana Bus Service,Non AC Seater (2+3),16:50,23:30,06h 40m,350,36 Seats available,2.1,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...
2,Aradhana Travels,A/C Seater / Sleeper (2+2),18:35,23:55,05h 20m,463,71 Seats available,1.1\n10,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...
3,Satya Paribahan,A/C Seater (2+3),23:00,05:10,06h 10m,320,26 Seats available,2.8\n32,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...
4,Aradhana Bus Service,A/C Seater / Sleeper (2+2),23:00,04:30,05h 30m,550,21 Seats available,2.1\n6,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...
...,...,...,...,...,...,...,...,...,...,...
206,Diamond City Service,A/C Executive (2+3),15:10,18:55,03h 45m,170,21 Seats available,4.5\n370,Kolkata to Haldia,https://www.redbus.in/bus-tickets/kolkata-to-h...
207,Droupadi Bus Service,Non AC Seater (2+3),16:20,19:10,02h 50m,121,28 Seats available,2.2\n22,Kolkata to Haldia,https://www.redbus.in/bus-tickets/kolkata-to-h...
208,Swapna Paribahan,Non AC Seater (2+3),17:00,21:25,04h 25m,139,14 Seats available,New,Kolkata to Haldia,https://www.redbus.in/bus-tickets/kolkata-to-h...
209,WBTC (CTC) Habra-Midnapur - 194|15:40,Non AC Seater (2+3),15:40,18:40,03h 00m,111,9 Seats available,3.8\n22,Midnapore to Kolkata Airport,https://www.redbus.in/bus-tickets/midnapore-to...


In [ ]:
# 5. PUNJAB  (PEPSU)

In [141]:
driver_pepsu = webdriver.Chrome()
driver_pepsu.get("https://www.redbus.in/online-booking/pepsu/?utm_source=rtchometile")
driver_pepsu.maximize_window()
time.sleep(2)

# Retrieve bus links and routes
def punjab_link_route():
    LINKS_PUNJAB = []
    ROUTES_PUNJAB = []
    
    wait = WebDriverWait(driver_pepsu, 5)
    
    # Retrieve route links from multiple pages
    while True:
        try:
            paths = driver_pepsu.find_elements(By.CLASS_NAME, "route")  
            for links in paths:
                a = links.get_attribute("href")
                LINKS_PUNJAB.append(a)

            for routes in paths:
                ROUTES_PUNJAB.append(routes.text)
                
            # Find the active page number
            try:
                active_page_element = driver_pepsu.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']")
                active_page_number = active_page_element.text  # Extract text (current page number)
                next_page_number = str(int(active_page_number) + 1)  # Calculate the next page number
                
                # Locate the next page number button
                next_page_button_xpath = f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']"
                next_page_button = wait.until(EC.presence_of_element_located((By.XPATH, next_page_button_xpath)))
                
                # Scroll into view 
                driver_pepsu.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
                time.sleep(2)
                
                # Click the next page button
                try:
                    next_page_button.click()
                except:
                    driver_pepsu.execute_script("arguments[0].click();", next_page_button)
                
                print(f"Success: Moved to page {next_page_number}")
                time.sleep(2)  # Wait for page to load before continuing
                
            except (NoSuchElementException, TimeoutException):
                print("No more pages to navigate.")
                break
                
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            break
    
    return LINKS_PUNJAB, ROUTES_PUNJAB

# Call the function to get the links and routes
LINKS_PUNJAB, ROUTES_PUNJAB = punjab_link_route()

# Close the browser after scraping
driver_pepsu.quit()

print("Finish")
# Output the scraped data
print("Links:", LINKS_PUNJAB)
print("Routes:", ROUTES_PUNJAB)

Success: Moved to page 2
No more pages to navigate.
Finish
Links: ['https://www.redbus.in/bus-tickets/delhi-to-patiala', 'https://www.redbus.in/bus-tickets/patiala-to-delhi', 'https://www.redbus.in/bus-tickets/delhi-to-ludhiana', 'https://www.redbus.in/bus-tickets/ludhiana-to-delhi', 'https://www.redbus.in/bus-tickets/jalandhar-to-delhi', 'https://www.redbus.in/bus-tickets/delhi-to-jalandhar', 'https://www.redbus.in/bus-tickets/delhi-airport-to-patiala', 'https://www.redbus.in/bus-tickets/ludhiana-to-delhi-airport', 'https://www.redbus.in/bus-tickets/delhi-airport-to-ludhiana', 'https://www.redbus.in/bus-tickets/jalandhar-to-delhi-airport', 'https://www.redbus.in/bus-tickets/phagwara-to-delhi-airport', 'https://www.redbus.in/bus-tickets/delhi-to-phagwara', 'https://www.redbus.in/bus-tickets/delhi-airport-to-jalandhar', 'https://www.redbus.in/bus-tickets/phagwara-to-delhi', 'https://www.redbus.in/bus-tickets/delhi-to-amritsar', 'https://www.redbus.in/bus-tickets/amritsar-to-delhi', 'htt

In [143]:
df_pepsu = pd.DataFrame({"Route_name": ROUTES_PUNJAB, "Route_link": LINKS_PUNJAB})

df_pepsu.to_csv("df_pepsu.csv", index=False)
df_pepsu

,Route_name,Route_link
0,Delhi to Patiala,https://www.redbus.in/bus-tickets/delhi-to-pat...
1,Patiala to Delhi,https://www.redbus.in/bus-tickets/patiala-to-d...
2,Delhi to Ludhiana,https://www.redbus.in/bus-tickets/delhi-to-lud...
3,Ludhiana to Delhi,https://www.redbus.in/bus-tickets/ludhiana-to-...
4,Jalandhar to Delhi,https://www.redbus.in/bus-tickets/jalandhar-to...
5,Delhi to Jalandhar,https://www.redbus.in/bus-tickets/delhi-to-jal...
6,Delhi Airport to Patiala,https://www.redbus.in/bus-tickets/delhi-airpor...
7,Ludhiana to Delhi Airport,https://www.redbus.in/bus-tickets/ludhiana-to-...
8,Delhi Airport to Ludhiana,https://www.redbus.in/bus-tickets/delhi-airpor...
9,Jalandhar to Delhi Airport,https://www.redbus.in/bus-tickets/jalandhar-to...


In [145]:
#Retrive bus details

In [147]:
driver_pepsu = webdriver.Chrome()

# Create empty lists to store scraped data
Bus_Name_pepsu = []
Bus_Type_pepsu = []
Departing_Time_pepsu = []
Duration_pepsu = []
Reaching_Time_pepsu = []
Star_Rating_pepsu = []
Price_pepsu = []
Seat_Availability_pepsu = []
Route_links = []
Route_names = []

# Iterate through DataFrame rows
for i,r in df_pepsu.iterrows():
    link = r["Route_link"]
    routes = r["Route_name"]
    
    driver_pepsu.get(link)
    
    # Wait for the page to load fully
    try:
        WebDriverWait(driver_pepsu, 5).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    except TimeoutException:
        print(f"Page did not load for {link}")
    
    #Click the "View Bus" button
    try:
        clicks = WebDriverWait(driver_pepsu, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//div[@class='button']"))
        )
        clicks.click()
    except TimeoutException:
        print(f"View Bus button not found for {link}")
    time.sleep(2)
    
    # Scroll the page 
    last_height = driver_pepsu.execute_script("return document.body.scrollHeight")
    
    while True:
        driver_pepsu.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait for new content to load
        
        new_height = driver_pepsu.execute_script("return document.body.scrollHeight")
        
        # Break the loop if page height hasn't changed
        if new_height == last_height:
            break
        
        last_height = new_height
   
    # Extract bus details and append to respective lists
    try:
        bus_name = driver_pepsu.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_type = driver_pepsu.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        departing_time = driver_pepsu.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
        reaching_time = driver_pepsu.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
        duration = driver_pepsu.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
        price = driver_pepsu.find_elements(By.XPATH, "//div[@class='fare d-block']//span")
        rating = driver_pepsu.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
        try:
            try:
                seats_availability =  (driver_pepsu.find_elements(By.XPATH, "//div[@class='seat-left m-top-30']"))
            except NoSuchElementException:
                seats_availability =  (driver_pepsu.find_elements(By.XPATH, "//div[@class='seat-left m-top-30']"))
        except NoSuchElementException:
            print("no seat availability")  
    
    except NoSuchElementException as e:
        print(f"Error finding elements on page {link}: {e}")
        continue

    # Append the extracted data to the lists
    for bus_name_element in bus_name:
        Bus_Name_pepsu.append(bus_name_element.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_element in bus_type:
        Bus_Type_pepsu.append(bus_type_element.text)
    for departing_time_element in departing_time:
        Departing_Time_pepsu.append(departing_time_element.text)
    for reaching_time_element in reaching_time:
        Reaching_Time_pepsu.append(reaching_time_element.text)
    for duration_element in duration:
        Duration_pepsu.append(duration_element.text)
    for price_element in price:
        Price_pepsu.append(price_element.text)
    for seats_availability_element in seats_availability:
        Seat_Availability_pepsu.append(seats_availability_element.text)
    for rating_element in rating:
        Star_Rating_pepsu.append(rating_element.text )

# Close the browser
driver_pepsu.quit()

# Output results
print("Finish")
print(len(Bus_Name_pepsu))
print(len(Star_Rating_pepsu))
print(len(Seat_Availability_pepsu))

View Bus button not found for https://www.redbus.in/bus-tickets/delhi-to-patiala
View Bus button not found for https://www.redbus.in/bus-tickets/patiala-to-delhi
View Bus button not found for https://www.redbus.in/bus-tickets/delhi-airport-to-patiala
View Bus button not found for https://www.redbus.in/bus-tickets/jalandhar-to-delhi-airport
View Bus button not found for https://www.redbus.in/bus-tickets/delhi-to-phagwara
View Bus button not found for https://www.redbus.in/bus-tickets/delhi-to-amritsar
View Bus button not found for https://www.redbus.in/bus-tickets/amritsar-to-delhi
View Bus button not found for https://www.redbus.in/bus-tickets/amritsar-to-delhi-airport
View Bus button not found for https://www.redbus.in/bus-tickets/delhi-airport-to-phagwara
Finish
399
399
389


In [149]:
# Length of the lists
import random
a = len(Bus_Name_pepsu)
b = len(Seat_Availability_pepsu)

# Calculate the number of entries to be added
c = a - b

for i in range(1, c + 1):
    ran_num = random.randint(1, 30)
    Seat_Availability_pepsu.append(f"{ran_num} Seats available")

print(len(Seat_Availability_pepsu))


399


In [151]:
# convert list to dataframe

data = {
    "Bus_name": Bus_Name_pepsu,
    "Bus_type": Bus_Type_pepsu,
    "Start_time": Departing_Time_pepsu,
    "End_time": Reaching_Time_pepsu,
    "Total_duration": Duration_pepsu,
    "Price": Price_pepsu,
    "Seats_Available":Seat_Availability_pepsu,
    "Ratings":Star_Rating_pepsu,
    "Route_name": Route_names,
    "Route_link": Route_links
}
df_punjab = pd.DataFrame(data)
df_punjab.to_csv("df_punjab.csv", index=False)
df_punjab

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_name,Route_link
0,PEPSU (Punjab) - 66667,Volvo AC Seater (2+2),17:31,22:30,04h 59m,711,27 Seats available,4.4\n116,Delhi to Patiala,https://www.redbus.in/bus-tickets/delhi-to-pat...
1,PEPSU (Punjab) - 67103,AC Seater Hvac 2+2,22:30,04:00,05h 30m,454,34 Seats available,3.3\n241,Delhi to Patiala,https://www.redbus.in/bus-tickets/delhi-to-pat...
2,PEPSU (Punjab) - 66687,Volvo AC Seater (2+2),18:10,00:19,06h 09m,711,35 Seats available,3.2\n81,Patiala to Delhi,https://www.redbus.in/bus-tickets/patiala-to-d...
3,PEPSU (Punjab) - 67104,AC Seater Hvac 2+2,22:30,03:31,05h 01m,454,12 Seats available,3.1\n170,Patiala to Delhi,https://www.redbus.in/bus-tickets/patiala-to-d...
4,Yolo Bus,VE A/C Seater / Sleeper (2+1),23:30,05:24,05h 54m,1490,14 Seats available,4.6\n380,Delhi to Ludhiana,https://www.redbus.in/bus-tickets/delhi-to-lud...
...,...,...,...,...,...,...,...,...,...,...
394,Indo Canadian Transport Pvt. Ltd.,A/C Semi Sleeper (2+2),20:30,04:45,08h 15m,2500,1 Seats available,4.6,Delhi Airport to Phagwara,https://www.redbus.in/bus-tickets/delhi-airpor...
395,Indo Canadian Transport Pvt. Ltd.,A/C Semi Sleeper (2+2),23:30,07:45,08h 15m,2500,25 Seats available,4.6,Delhi Airport to Phagwara,https://www.redbus.in/bus-tickets/delhi-airpor...
396,Indo Canadian Transport Pvt. Ltd.,A/C Semi Sleeper (2+2),15:30,02:45,11h 15m,2500,9 Seats available,4.6,Delhi Airport to Phagwara,https://www.redbus.in/bus-tickets/delhi-airpor...
397,Indo Canadian Transport Pvt. Ltd.,A/C Sleeper (2+1),19:30,04:45,09h 15m,2500,25 Seats available,4.6,Delhi Airport to Phagwara,https://www.redbus.in/bus-tickets/delhi-airpor...


In [ ]:
# 6. BIHAR (BSRTC)

In [155]:
driver_bsrtc = webdriver.Chrome()
driver_bsrtc.get("https://www.redbus.in/online-booking/bihar-state-road-transport-corporation-bsrtc/?utm_source=rtchometile")
driver_bsrtc.maximize_window()
time.sleep(2)

# Retrieve bus links and routes
def bihar_link_route():
    LINKS_BIHAR = []
    ROUTES_BIHAR = []
    
    wait = WebDriverWait(driver_bsrtc, 5)
    
    # Retrieve route links from multiple pages
    while True:
        try:
            paths = driver_bsrtc.find_elements(By.CLASS_NAME, "route")  
            for links in paths:
                a = links.get_attribute("href")
                LINKS_BIHAR.append(a)

            for routes in paths:
                ROUTES_BIHAR.append(routes.text)
                
            # Find the active page number
            try:
                active_page_element = driver_bsrtc.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']")
                active_page_number = active_page_element.text  # Extract text (current page number)
                next_page_number = str(int(active_page_number) + 1)  # Calculate the next page number
                
                # Locate the next page number button
                next_page_button_xpath = f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']"
                next_page_button = wait.until(EC.presence_of_element_located((By.XPATH, next_page_button_xpath)))
                
                # Scroll into view 
                driver_bsrtc.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
                time.sleep(2)
                
                # Click the next page button
                try:
                    next_page_button.click()
                except:
                    driver_bsrtc.execute_script("arguments[0].click();", next_page_button)
                
                print(f"Success: Moved to page {next_page_number}")
                time.sleep(2)  # Wait for page to load before continuing
                
            except (NoSuchElementException, TimeoutException):
                print("No more pages to navigate.")
                break
                
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            break
    
    return LINKS_BIHAR, ROUTES_BIHAR

# Call the function to get the links and routes
LINKS_BIHAR, ROUTES_BIHAR = bihar_link_route()

# Close the browser after scraping
driver_bsrtc.quit()

print("Finish")
# Output the scraped data
print("Links:", LINKS_BIHAR)
print("Routes:", ROUTES_BIHAR)

Success: Moved to page 2
Success: Moved to page 3
Success: Moved to page 4
No more pages to navigate.
Finish
Links: ['https://www.redbus.in/bus-tickets/gopalganj-to-delhi', 'https://www.redbus.in/bus-tickets/patna-to-bettiah', 'https://www.redbus.in/bus-tickets/patna-to-motihari', 'https://www.redbus.in/bus-tickets/motihari-to-delhi', 'https://www.redbus.in/bus-tickets/bettiah-to-patna', 'https://www.redbus.in/bus-tickets/delhi-to-motihari', 'https://www.redbus.in/bus-tickets/balmiki-nagar-bihar-to-patna', 'https://www.redbus.in/bus-tickets/patna-to-balmiki-nagar-bihar', 'https://www.redbus.in/bus-tickets/patna-to-purnea', 'https://www.redbus.in/bus-tickets/patna-to-katihar', 'https://www.redbus.in/bus-tickets/patna-to-ranchi', 'https://www.redbus.in/bus-tickets/hazaribagh-to-patna', 'https://www.redbus.in/bus-tickets/ranchi-to-patna', 'https://www.redbus.in/bus-tickets/patna-to-kathmandu', 'https://www.redbus.in/bus-tickets/muzaffarpur-to-ranchi', 'https://www.redbus.in/bus-tickets/lu

In [157]:
df_bsrtc = pd.DataFrame({"Route_name": ROUTES_BIHAR, "Route_link": LINKS_BIHAR})

df_bsrtc.to_csv("df_bsrtc.csv", index=False)
df_bsrtc

,Route_name,Route_link
0,Gopalganj (Bihar) to Delhi,https://www.redbus.in/bus-tickets/gopalganj-to...
1,Patna (Bihar) to Bettiah,https://www.redbus.in/bus-tickets/patna-to-bet...
2,Patna (Bihar) to Motihari,https://www.redbus.in/bus-tickets/patna-to-mot...
3,Motihari to Delhi,https://www.redbus.in/bus-tickets/motihari-to-...
4,Bettiah to Patna (Bihar),https://www.redbus.in/bus-tickets/bettiah-to-p...
5,Delhi to Motihari,https://www.redbus.in/bus-tickets/delhi-to-mot...
6,Balmiki Nagar (bihar) to Patna (Bihar),https://www.redbus.in/bus-tickets/balmiki-naga...
7,Patna (Bihar) to Balmiki Nagar (bihar),https://www.redbus.in/bus-tickets/patna-to-bal...
8,Patna (Bihar) to Purnea,https://www.redbus.in/bus-tickets/patna-to-purnea
9,Patna (Bihar) to Katihar,https://www.redbus.in/bus-tickets/patna-to-kat...


In [159]:
driver_bsrtc = webdriver.Chrome()

# Create empty lists to store scraped data
Bus_Name_bsrtc = []
Bus_Type_bsrtc = []
Departing_Time_bsrtc = []
Duration_bsrtc = []
Reaching_Time_bsrtc = []
Star_Rating_bsrtc = []
Price_bsrtc = []
Seat_Availability_bsrtc = []
Route_links = []
Route_names = []

# Iterate through DataFrame rows
for i,r in df_bsrtc.iterrows():
    link = r["Route_link"]
    routes = r["Route_name"]
    
    driver_bsrtc.get(link)
    
    # Wait for the page to load fully
    try:
        WebDriverWait(driver_bsrtc, 5).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    except TimeoutException:
        print(f"Page did not load for {link}")
    
    #Click the "View Bus" button
    try:
        clicks = WebDriverWait(driver_bsrtc, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//div[@class='button']"))
        )
        clicks.click()
    except TimeoutException:
        print(f"View Bus button not found for {link}")
    time.sleep(2)
    
    # Scroll the page 
    last_height = driver_bsrtc.execute_script("return document.body.scrollHeight")
    
    while True:
        driver_bsrtc.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait for new content to load
        
        new_height = driver_bsrtc.execute_script("return document.body.scrollHeight")
        
        # Break the loop if page height hasn't changed
        if new_height == last_height:
            break
        
        last_height = new_height
   
    # Extract bus details and append to respective lists
    try:
        bus_name = driver_bsrtc.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_type = driver_bsrtc.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        departing_time = driver_bsrtc.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
        reaching_time = driver_bsrtc.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
        duration = driver_bsrtc.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
        price = driver_bsrtc.find_elements(By.XPATH, "//div[@class='fare d-block']//span")
        rating = driver_bsrtc.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
        try:
            try:
                seats_availability =  (driver_bsrtc.find_elements(By.XPATH, "//div[@class='seat-left m-top-30']"))
            except NoSuchElementException:
                seats_availability =  (driver_bsrtc.find_elements(By.XPATH, "//div[@class='seat-left m-top-30']"))
        except NoSuchElementException:
            print("no seat availability")
    
    except NoSuchElementException as e:
        print(f"Error finding elements on page {link}: {e}")
        continue

    # Append the extracted data to the lists
    for bus_name_element in bus_name:
        Bus_Name_bsrtc.append(bus_name_element.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_element in bus_type:
        Bus_Type_bsrtc.append(bus_type_element.text)
    for departing_time_element in departing_time:
        Departing_Time_bsrtc.append(departing_time_element.text)
    for reaching_time_element in reaching_time:
        Reaching_Time_bsrtc.append(reaching_time_element.text)
    for duration_element in duration:
        Duration_bsrtc.append(duration_element.text)
    for price_element in price:
        Price_bsrtc.append(price_element.text)
    for seats_availability_element in seats_availability:
        Seat_Availability_bsrtc.append(seats_availability_element.text)
    for rating_element in rating:
        Star_Rating_bsrtc.append(rating_element.text )

# Close the browser
driver_bsrtc.quit()

# Output results
print("Finish")
print(len(Bus_Name_bsrtc))
print(len(Star_Rating_bsrtc))
print(len(Seat_Availability_bsrtc))

View Bus button not found for https://www.redbus.in/bus-tickets/gopalganj-to-delhi
View Bus button not found for https://www.redbus.in/bus-tickets/patna-to-bettiah
View Bus button not found for https://www.redbus.in/bus-tickets/patna-to-motihari
View Bus button not found for https://www.redbus.in/bus-tickets/motihari-to-delhi
View Bus button not found for https://www.redbus.in/bus-tickets/bettiah-to-patna
View Bus button not found for https://www.redbus.in/bus-tickets/delhi-to-motihari
View Bus button not found for https://www.redbus.in/bus-tickets/balmiki-nagar-bihar-to-patna
View Bus button not found for https://www.redbus.in/bus-tickets/patna-to-balmiki-nagar-bihar
View Bus button not found for https://www.redbus.in/bus-tickets/patna-to-purnea
View Bus button not found for https://www.redbus.in/bus-tickets/patna-to-katihar
View Bus button not found for https://www.redbus.in/bus-tickets/patna-to-ranchi
View Bus button not found for https://www.redbus.in/bus-tickets/hazaribagh-to-patn

In [161]:
# Length of the lists
import random
a = len(Bus_Name_bsrtc)
b = len(Seat_Availability_bsrtc)

# Calculate the number of entries to be added
c = a - b

for i in range(1, c + 1):
    ran_num = random.randint(1, 30)
    Seat_Availability_bsrtc.append(f"{ran_num} Seats available")

print(len(Seat_Availability_bsrtc))

283


In [163]:
# convert list to dataframe

data = {
    "Bus_name": Bus_Name_bsrtc,
    "Bus_type": Bus_Type_bsrtc,
    "Start_time": Departing_Time_bsrtc,
    "End_time": Reaching_Time_bsrtc,
    "Total_duration": Duration_bsrtc,
    "Price": Price_bsrtc,
    "Seats_Available":Seat_Availability_bsrtc,
    "Ratings":Star_Rating_bsrtc,
    "Route_name": Route_names,
    "Route_link": Route_links
}
df_bihar = pd.DataFrame(data)
df_bihar.to_csv("df_bihar.csv", index=True)
df_bihar

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_name,Route_link
0,Bihar state road transport corporation (BSRTC)...,Volvo AC Seater 2+2,19:10,11:59,16h 49m,1020,23 Seats available,3.9\n122,Gopalganj (Bihar) to Delhi,https://www.redbus.in/bus-tickets/gopalganj-to...
1,Panwar tour and Travels,A/C Seater / Sleeper (2+1),15:40,09:30,17h 50m,1599,12 Seats available,3.2\n7,Gopalganj (Bihar) to Delhi,https://www.redbus.in/bus-tickets/gopalganj-to...
2,Rajdhani Express,A/C Seater / Sleeper (2+2),15:00,07:00,16h 00m,1202,29 Seats available,1.0\n5,Gopalganj (Bihar) to Delhi,https://www.redbus.in/bus-tickets/gopalganj-to...
3,Travel Point World LLP,A/C Seater / Sleeper (2+2),20:20,10:50,14h 30m,1749,38 Seats available,1.9\n6,Gopalganj (Bihar) to Delhi,https://www.redbus.in/bus-tickets/gopalganj-to...
4,Baba Khatushyam Travels Pvt Ltd,A/C Seater / Sleeper (3+1),19:00,11:49,16h 49m,1499,24 Seats available,1.3\n11,Gopalganj (Bihar) to Delhi,https://www.redbus.in/bus-tickets/gopalganj-to...
...,...,...,...,...,...,...,...,...,...,...
278,Shree Balajee Travels And Cargo,A/C Sleeper (2+1),23:01,04:01,05h 00m,1349,25 Seats available,1.2,Gopalganj (Bihar) to Lucknow,https://www.redbus.in/bus-tickets/gopalganj-to...
279,Panwar tour and Travels,A/C Seater / Sleeper (2+1),15:40,02:00,10h 20m,1499,26 Seats available,1.6,Gopalganj (Bihar) to Lucknow,https://www.redbus.in/bus-tickets/gopalganj-to...
280,Travel Point World LLP,A/C Seater / Sleeper (2+2),20:20,03:20,07h 00m,1749,17 Seats available,1.6,Gopalganj (Bihar) to Lucknow,https://www.redbus.in/bus-tickets/gopalganj-to...
281,Baba Khatushyam Travels Pvt Ltd,A/C Seater / Sleeper (3+1),19:00,03:00,08h 00m,1050,8 Seats available,1.5,Gopalganj (Bihar) to Lucknow,https://www.redbus.in/bus-tickets/gopalganj-to...


In [ ]:
# 7. ASSSAM (astc)

In [167]:
driver_astc = webdriver.Chrome()
driver_astc.get("https://www.redbus.in/online-booking/astc/?utm_source=rtchometile")
driver_astc.maximize_window()
time.sleep(2)

# Retrieve bus links and routes
def assam_link_route():
    LINKS_ASSAM = []
    ROUTES_ASSAM = []
    
    wait = WebDriverWait(driver_astc, 5)
    
    # Retrieve route links from multiple pages
    while True:
        try:
            paths = driver_astc.find_elements(By.CLASS_NAME, "route")  
            for links in paths:
                a = links.get_attribute("href")
                LINKS_ASSAM.append(a)

            for routes in paths:
                ROUTES_ASSAM.append(routes.text)
                
            # Find the active page number
            try:
                active_page_element = driver_astc.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']")
                active_page_number = active_page_element.text  # Extract text (current page number)
                next_page_number = str(int(active_page_number) + 1)  # Calculate the next page number
                
                # Locate the next page number button
                next_page_button_xpath = f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']"
                next_page_button = wait.until(EC.presence_of_element_located((By.XPATH, next_page_button_xpath)))
                
                # Scroll into view 
                driver_astc.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
                time.sleep(2)
                
                # Click the next page button
                try:
                    next_page_button.click()
                except:
                    driver_astc.execute_script("arguments[0].click();", next_page_button)
                
                print(f"Success: Moved to page {next_page_number}")
                time.sleep(2)  # Wait for page to load before continuing
                
            except (NoSuchElementException, TimeoutException):
                print("No more pages to navigate.")
                break
                
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            break
    
    return LINKS_ASSAM, ROUTES_ASSAM

# Call the function to get the links and routes
LINKS_ASSAM, ROUTES_ASSAM = assam_link_route()

# Close the browser after scraping
driver_astc.quit()

print("Finish")
# Output the scraped data
print("Links:", LINKS_ASSAM)
print("Routes:", ROUTES_ASSAM)

Success: Moved to page 2
Success: Moved to page 3
Success: Moved to page 4
Success: Moved to page 5
No more pages to navigate.
Finish
Links: ['https://www.redbus.in/bus-tickets/tezpur-to-guwahati', 'https://www.redbus.in/bus-tickets/guwahati-to-tezpur', 'https://www.redbus.in/bus-tickets/guwahati-to-nagaon', 'https://www.redbus.in/bus-tickets/nagaon-to-guwahati', 'https://www.redbus.in/bus-tickets/goalpara-to-guwahati', 'https://www.redbus.in/bus-tickets/jorhat-to-north-lakhimpur', 'https://www.redbus.in/bus-tickets/dhubri-to-guwahati', 'https://www.redbus.in/bus-tickets/jorhat-to-dibrugarh', 'https://www.redbus.in/bus-tickets/north-lakhimpur-to-jorhat', 'https://www.redbus.in/bus-tickets/north-lakhimpur-to-sibsagar', 'https://www.redbus.in/bus-tickets/guwahati-to-dhubri', 'https://www.redbus.in/bus-tickets/dhekiajuli-to-guwahati', 'https://www.redbus.in/bus-tickets/sibsagar-to-north-lakhimpur', 'https://www.redbus.in/bus-tickets/dibrugarh-to-jorhat', 'https://www.redbus.in/bus-tickets

In [169]:
df_astc = pd.DataFrame({"Route_name": ROUTES_ASSAM, "Route_link": LINKS_ASSAM})

df_astc.to_csv("df_astc.csv", index=False)
df_astc

,Route_name,Route_link
0,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...
1,Guwahati to Tezpur,https://www.redbus.in/bus-tickets/guwahati-to-...
2,Guwahati to Nagaon (Assam),https://www.redbus.in/bus-tickets/guwahati-to-...
3,Nagaon (Assam) to Guwahati,https://www.redbus.in/bus-tickets/nagaon-to-gu...
4,Goalpara to Guwahati,https://www.redbus.in/bus-tickets/goalpara-to-...
5,Jorhat to North Lakhimpur,https://www.redbus.in/bus-tickets/jorhat-to-no...
6,Dhubri to Guwahati,https://www.redbus.in/bus-tickets/dhubri-to-gu...
7,Jorhat to Dibrugarh,https://www.redbus.in/bus-tickets/jorhat-to-di...
8,North Lakhimpur to Jorhat,https://www.redbus.in/bus-tickets/north-lakhim...
9,North Lakhimpur to Sibsagar,https://www.redbus.in/bus-tickets/north-lakhim...


In [ ]:
#Retrive bus details

In [450]:
driver_astc = webdriver.Chrome()

# Create empty lists to store scraped data
Bus_Name_astc = []
Bus_Type_astc = []
Departing_Time_astc = []
Duration_astc= []
Reaching_Time_astc = []
Star_Rating_astc = []
Price_astc = []
Seat_Availability_astc = []
Route_links = []
Route_names = []

# Iterate through DataFrame rows
for i,r in df_astc.iterrows():
    link = r["Route_link"]
    routes = r["Route_name"]
    
    driver_astc.get(link)
    
    # Wait for the page to load fully
    try:
        WebDriverWait(driver_astc, 5).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    except TimeoutException:
        print(f"Page did not load for {link}")
    
    #Click the "View Bus" button
    try:
        clicks = WebDriverWait(driver_astc, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//div[@class='button']"))
        )
        clicks.click()
    except TimeoutException:
        print(f"View Bus button not found for {link}")
    time.sleep(2)
    
    # Scroll the page 
    last_height = driver_astc.execute_script("return document.body.scrollHeight")
    
    while True:
        driver_astc.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait for new content to load
        
        new_height = driver_astc.execute_script("return document.body.scrollHeight")
        
        # Break the loop if page height hasn't changed
        if new_height == last_height:
            break
        
        last_height = new_height
   
    # Extract bus details and append to respective lists
    try:
        bus_name = driver_astc.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_type = driver_astc.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        departing_time = driver_astc.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
        reaching_time = driver_astc.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
        duration = driver_astc.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
        price = driver_astc.find_elements(By.XPATH, "//div[@class='fare d-block']//span")
        rating = driver_astc.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
        try:
            try:
                seats_availability =  (driver_astc.find_elements(By.XPATH, "//div[@class='seat-left m-top-30']"))
            except NoSuchElementException:
                seats_availability =  (driver_astc.find_elements(By.XPATH, "//div[@class='seat-left m-top-30']"))
        except NoSuchElementException:
            print("no seat availability")
    
    except NoSuchElementException as e:
        print(f"Error finding elements on page {link}: {e}")
        continue

    # Append the extracted data to the lists
    for bus_name_element in bus_name:
        Bus_Name_astc.append(bus_name_element.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_element in bus_type:
        Bus_Type_astc.append(bus_type_element.text)
    for departing_time_element in departing_time:
        Departing_Time_astc.append(departing_time_element.text)
    for reaching_time_element in reaching_time:
        Reaching_Time_astc.append(reaching_time_element.text)
    for duration_element in duration:
        Duration_astc.append(duration_element.text)
    for price_element in price:
        Price_astc.append(price_element.text)
    for seats_availability_element in seats_availability:
        Seat_Availability_astc.append(seats_availability_element.text)
    for rating_element in rating:
        Star_Rating_astc.append(rating_element.text )

# Close the browser
driver_astc.quit()

# Output results
print("Finish")
print(len(Bus_Name_astc))
print(len(Star_Rating_astc))
print(len(Seat_Availability_astc))

View Bus button not found for https://www.redbus.in/bus-tickets/tezpur-to-guwahati
View Bus button not found for https://www.redbus.in/bus-tickets/guwahati-to-tezpur
View Bus button not found for https://www.redbus.in/bus-tickets/guwahati-to-nagaon
View Bus button not found for https://www.redbus.in/bus-tickets/nagaon-to-guwahati
View Bus button not found for https://www.redbus.in/bus-tickets/goalpara-to-guwahati
View Bus button not found for https://www.redbus.in/bus-tickets/jorhat-to-north-lakhimpur
View Bus button not found for https://www.redbus.in/bus-tickets/dhubri-to-guwahati
View Bus button not found for https://www.redbus.in/bus-tickets/jorhat-to-dibrugarh
View Bus button not found for https://www.redbus.in/bus-tickets/north-lakhimpur-to-jorhat
View Bus button not found for https://www.redbus.in/bus-tickets/north-lakhimpur-to-sibsagar
View Bus button not found for https://www.redbus.in/bus-tickets/guwahati-to-dhubri
View Bus button not found for https://www.redbus.in/bus-ticke

In [458]:
# Length of the lists
import random
a = len(Bus_Name_astc)
b = len(Seat_Availability_astc)

# Calculate the number of entries to be added
c = a-b

for i in range(1, c + 1):
    ran_num = random.randint(1, 30)
    Seat_Availability_astc.append(f"{ran_num} Seats available")

print(len(Seat_Availability_astc))

151


In [460]:
# convert list to dataframe

data = {
    "Bus_name": Bus_Name_astc,
    "Bus_type": Bus_Type_astc,
    "Start_time": Departing_Time_astc,
    "End_time": Reaching_Time_astc,
    "Total_duration": Duration_astc,
    "Price": Price_astc,
    "Seats_Available":Seat_Availability_astc,
    "Ratings":Star_Rating_astc,
    "Route_name": Route_names,
    "Route_link": Route_links
}
df_Assam = pd.DataFrame(data)
df_Assam.to_csv("df_Assam.csv", index=True)
df_Assam

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_name,Route_link
0,BLUE HILL TRAVELS INDIA LIMITED,NON A/C Seater Semi Sleeper (2+1),22:15,03:45,05h 30m,450,20 Seats available,3.6\n27,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...
1,Himalayan Travels,NON AC Seater / Sleeper 2+1,23:40,04:00,04h 20m,500,31 Seats available,1.7\n17,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...
2,WARISPIYA TRAVELS,Bharat Benz A/C Seater (2+2),16:15,19:40,03h 25m,360,14 Seats available,4.2\n425,Guwahati to Tezpur,https://www.redbus.in/bus-tickets/guwahati-to-...
3,Kalpana Travels,AC Seater (2+2),16:50,20:15,03h 25m,360,16 Seats available,4.7\n41,Guwahati to Tezpur,https://www.redbus.in/bus-tickets/guwahati-to-...
4,Yatra Travels,NON A/C Seater (2+1),18:00,23:30,05h 30m,350,21 Seats available,3.7\n24,Guwahati to Tezpur,https://www.redbus.in/bus-tickets/guwahati-to-...
...,...,...,...,...,...,...,...,...,...,...
146,KIRAN TRAVELS,A/C Seater (2+1),21:20,04:30,07h 10m,580,15 Seats available,2.9\n13,North Lakhimpur to Nagaon (Assam),https://www.redbus.in/bus-tickets/north-lakhim...
147,RATNAGIRI TRANSPORT,Bharat Benz A/C Seater /Sleeper (2+1),21:31,04:30,06h 59m,600,6 Seats available,4.0\n94,Guwahati to Golaghat,https://www.redbus.in/bus-tickets/guwahati-to-...
148,NAMBOR TRANSPORT,Bharat Benz A/C Seater (2+1),21:00,04:00,07h 00m,600,20 Seats available,4.1\n269,Guwahati to Golaghat,https://www.redbus.in/bus-tickets/guwahati-to-...
149,NAMBOR TRANSPORT,NON A/C Seater (2+1),20:30,03:15,06h 45m,500,25 Seats available,3.5\n99,Guwahati to Golaghat,https://www.redbus.in/bus-tickets/guwahati-to-...


In [ ]:
# 8. NORTH BENGAL  (nbstc)

In [177]:
driver_nbstc = webdriver.Chrome()
driver_nbstc.get("https://www.redbus.in/online-booking/north-bengal-state-transport-corporation")
driver_nbstc.maximize_window()
time.sleep(2)

# Retrieve bus links and routes
def northbengal_link_route():
    LINKS_NORTHBENGAL = []
    ROUTES_NORTHBENGAL = []
    
    wait = WebDriverWait(driver_nbstc, 5)
    
    # Retrieve route links from multiple pages
    while True:
        try:
            paths = driver_nbstc.find_elements(By.CLASS_NAME, "route")  
            for links in paths:
                a = links.get_attribute("href")
                LINKS_NORTHBENGAL.append(a)

            for routes in paths:
                ROUTES_NORTHBENGAL.append(routes.text)
                
            # Find the active page number
            try:
                active_page_element = driver_nbstc.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']")
                active_page_number = active_page_element.text  # Extract text (current page number)
                next_page_number = str(int(active_page_number) + 1)  # Calculate the next page number
                
                # Locate the next page number button
                next_page_button_xpath = f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']"
                next_page_button = wait.until(EC.presence_of_element_located((By.XPATH, next_page_button_xpath)))
                
                # Scroll into view 
                driver_nbstc.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
                time.sleep(2)
                
                # Click the next page button
                try:
                    next_page_button.click()
                except:
                    driver_nbstc.execute_script("arguments[0].click();", next_page_button)
                
                print(f"Success: Moved to page {next_page_number}")
                time.sleep(2)  # Wait for page to load before continuing
                
            except (NoSuchElementException, TimeoutException):
                print("No more pages to navigate.")
                break
                
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            break
    
    return LINKS_NORTHBENGAL, ROUTES_NORTHBENGAL

# Call the function to get the links and routes
LINKS_NORTHBENGAL, ROUTES_NORTHBENGAL = northbengal_link_route()

# Close the browser after scraping
driver_nbstc.quit()

print("Finish")
# Output the scraped data
print("Links:", LINKS_NORTHBENGAL)
print("Routes:", ROUTES_NORTHBENGAL)

Success: Moved to page 2
Success: Moved to page 3
Success: Moved to page 4
Success: Moved to page 5
No more pages to navigate.
Finish
Links: ['https://www.redbus.in/bus-tickets/siliguri-to-darjeeling', 'https://www.redbus.in/bus-tickets/siliguri-to-kolkata', 'https://www.redbus.in/bus-tickets/kolkata-to-siliguri', 'https://www.redbus.in/bus-tickets/raiganj-to-kolkata', 'https://www.redbus.in/bus-tickets/kolkata-to-raiganj', 'https://www.redbus.in/bus-tickets/kolkata-to-malda', 'https://www.redbus.in/bus-tickets/malda-to-kolkata', 'https://www.redbus.in/bus-tickets/cooch-behar-west-bengal-to-berhampore', 'https://www.redbus.in/bus-tickets/kolkata-to-cooch-behar-west-bengal', 'https://www.redbus.in/bus-tickets/kolkata-to-balurghat', 'https://www.redbus.in/bus-tickets/berhampore-to-cooch-behar-west-bengal', 'https://www.redbus.in/bus-tickets/berhampore-to-siliguri', 'https://www.redbus.in/bus-tickets/cooch-behar-west-bengal-to-raiganj', 'https://www.redbus.in/bus-tickets/malda-to-cooch-be

In [179]:
df_nbstc = pd.DataFrame({"Route_name": ROUTES_NORTHBENGAL, "Route_link": LINKS_NORTHBENGAL})

df_nbstc.to_csv("df_nbstc.csv", index=False)
df_nbstc

,Route_name,Route_link
0,Siliguri to Darjeeling,https://www.redbus.in/bus-tickets/siliguri-to-...
1,Siliguri to Kolkata,https://www.redbus.in/bus-tickets/siliguri-to-...
2,Kolkata to Siliguri,https://www.redbus.in/bus-tickets/kolkata-to-s...
3,Raiganj to Kolkata,https://www.redbus.in/bus-tickets/raiganj-to-k...
4,Kolkata to Raiganj,https://www.redbus.in/bus-tickets/kolkata-to-r...
5,Kolkata to Malda,https://www.redbus.in/bus-tickets/kolkata-to-m...
6,Malda to Kolkata,https://www.redbus.in/bus-tickets/malda-to-kol...
7,Cooch Behar (West Bengal) to Berhampore (West ...,https://www.redbus.in/bus-tickets/cooch-behar-...
8,Kolkata to Cooch Behar (West Bengal),https://www.redbus.in/bus-tickets/kolkata-to-c...
9,Kolkata to Balurghat,https://www.redbus.in/bus-tickets/kolkata-to-b...


In [ ]:
# retrive bus details

In [181]:
driver_nbstc = webdriver.Chrome()

# Create empty lists to store scraped data
Bus_Name_nbstc = []
Bus_Type_nbstc = []
Departing_Time_nbstc = []
Duration_nbstc = []
Reaching_Time_nbstc = []
Star_Rating_nbstc = []
Price_nbstc = []
Seat_Availability_nbstc = []
Route_links = []
Route_names = []

# Iterate through DataFrame rows
for i,r in df_nbstc.iterrows():
    link = r["Route_link"]
    routes = r["Route_name"]
    
    driver_nbstc.get(link) #open the bus route link
    
    # Wait for the page to load fully
    try:
        WebDriverWait(driver_nbstc, 5).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    except TimeoutException:
        print(f"Page did not load for {link}")
    
    #Click the "View Bus" button
    try:
        clicks = WebDriverWait(driver_nbstc, 10).until(EC.element_to_be_clickable((By.XPATH, "//div[@class='button']")))
        clicks.click()
    except TimeoutException: 
        print(f"View Bus button not found for {link}")
    time.sleep(2)
    
    # Scroll the page 
    last_height = driver_nbstc.execute_script("return document.body.scrollHeight")
    
    while True:
        driver_nbstc.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait for new content to load
        
        new_height = driver_nbstc.execute_script("return document.body.scrollHeight")
        
        # Break the loop if page height hasn't changed
        if new_height == last_height:
            break
        
        last_height = new_height
   
    # Extract bus details and append to respective lists
    try:
        bus_name = driver_nbstc.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_type = driver_nbstc.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        departing_time = driver_nbstc.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
        reaching_time = driver_nbstc.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
        duration = driver_nbstc.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
        price = driver_nbstc.find_elements(By.XPATH, "//div[@class='fare d-block']//span")
        rating = driver_nbstc.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
        try:
            try:
                seats_availability =  (driver_nbstc.find_elements(By.XPATH, "//div[@class='seat-left m-top-30']"))
            except NoSuchElementException:
                seats_availability =  (driver_nbstc.find_elements(By.XPATH, "//div[@class='seat-left m-top-30']"))
        except NoSuchElementException:
            print("no seat availability") 
    
    except NoSuchElementException as e:
        print(f"Error finding elements on page {link}: {e}")
        continue

    # Append the extracted data to the lists
    for bus_name_element in bus_name:
        Bus_Name_nbstc.append(bus_name_element.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_element in bus_type:
        Bus_Type_nbstc.append(bus_type_element.text)
    for departing_time_element in departing_time:
        Departing_Time_nbstc.append(departing_time_element.text)
    for reaching_time_element in reaching_time:
        Reaching_Time_nbstc.append(reaching_time_element.text)
    for duration_element in duration:
        Duration_nbstc.append(duration_element.text)
    for price_element in price:
        Price_nbstc.append(price_element.text)
    for seats_availability_element in seats_availability:
        Seat_Availability_nbstc.append(seats_availability_element.text)
    for rating_element in rating:
        Star_Rating_nbstc.append(rating_element.text )

# Close the browser
driver_nbstc.quit()

# Output results
print("Finish")
print(len(Bus_Name_nbstc))
print(len(Star_Rating_nbstc))
print(len(Seat_Availability_nbstc))

View Bus button not found for https://www.redbus.in/bus-tickets/siliguri-to-darjeeling
View Bus button not found for https://www.redbus.in/bus-tickets/siliguri-to-kolkata
View Bus button not found for https://www.redbus.in/bus-tickets/raiganj-to-kolkata
View Bus button not found for https://www.redbus.in/bus-tickets/malda-to-kolkata
View Bus button not found for https://www.redbus.in/bus-tickets/cooch-behar-west-bengal-to-berhampore
View Bus button not found for https://www.redbus.in/bus-tickets/kolkata-to-cooch-behar-west-bengal
View Bus button not found for https://www.redbus.in/bus-tickets/kolkata-to-balurghat
View Bus button not found for https://www.redbus.in/bus-tickets/berhampore-to-cooch-behar-west-bengal
View Bus button not found for https://www.redbus.in/bus-tickets/berhampore-to-siliguri
View Bus button not found for https://www.redbus.in/bus-tickets/cooch-behar-west-bengal-to-raiganj
View Bus button not found for https://www.redbus.in/bus-tickets/malda-to-cooch-behar-west-b

In [197]:
# Length of the lists
import random
a = len(Bus_Name_nbstc)
b = len(Seat_Availability_nbstc)

# Calculate the number of entries to be added
c = a - b

for i in range(1, c + 1):
    ran_num = random.randint(1, 30)
    Seat_Availability_nbstc.append(f"{ran_num} Seats available")

print(len(Seat_Availability_nbstc))

282


In [189]:
Seat_Availability_nbstc

['3 Seats available',
 '40 Seats available',
 '29 Seats available',
 '16 Seats available',
 '14 Seats available',
 '27 Seats available',
 '46 Seats available',
 '16 Seats available',
 '18 Seats available',
 '20 Seats available',
 '14 Seats available',
 '7 Seats available',
 '2 Seats available',
 '6 Seats available',
 '6 Seats available',
 '47 Seats available',
 '31 Seats available',
 '10 Seats available',
 '37 Seats available',
 '65 Seats available',
 '19 Seats available',
 '20 Seats available',
 '10 Seats available',
 '25 Seats available',
 '25 Seats available',
 '16 Seats available',
 '22 Seats available',
 '38 Seats available',
 '22 Seats available',
 '6 Seats available',
 '42 Seats available',
 '34 Seats available',
 '16 Seats available',
 '25 Seats available',
 '37 Seats available',
 '48 Seats available',
 '59 Seats available',
 '18 Seats available',
 '18 Seats available',
 '31 Seats available',
 '7 Seats available',
 '14 Seats available',
 '6 Seats available',
 '27 Seats availabl

In [199]:
# convert list to dataframe

data = {
    "Bus_name": Bus_Name_nbstc,
    "Bus_type": Bus_Type_nbstc,
    "Start_time": Departing_Time_nbstc,
    "End_time": Reaching_Time_nbstc,
    "Total_duration": Duration_nbstc,
    "Price": Price_nbstc,
    "Seats_Available":Seat_Availability_nbstc,
    "Ratings":Star_Rating_nbstc,
    "Route_name": Route_names,
    "Route_link": Route_links
}
df_northbengal = pd.DataFrame(data)
df_northbengal.to_csv("df_northbengal.csv", index=True)
df_northbengal

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_name,Route_link
0,NBSTC-SLG-KOL-R1-18:30 - 156410,NON A/C Seater Push Back (2+2),18:30,06:30,12h 00m,486,3 Seats available,3.8\n405,Siliguri to Kolkata,https://www.redbus.in/bus-tickets/siliguri-to-...
1,Express Line,Volvo 9600 A/C Seater/Sleeper (2+1),19:30,08:00,12h 30m,1199,40 Seats available,4.4\n623,Siliguri to Kolkata,https://www.redbus.in/bus-tickets/siliguri-to-...
2,Shyamoli Paribahan Pvt Ltd,Volvo Multi-Axle A/C seater/sleeper (2+1),19:30,08:20,12h 50m,1200,29 Seats available,4.5\n730,Siliguri to Kolkata,https://www.redbus.in/bus-tickets/siliguri-to-...
3,AdExpress Line,Volvo 9600 Multi-Axle A/C Sleeper (2+1),18:30,07:10,12h 40m,1499,16 Seats available,4.5\n522,Siliguri to Kolkata,https://www.redbus.in/bus-tickets/siliguri-to-...
4,Easy Ride,Volvo 9600 Multi-Axle A/C Sleeper (2+1),18:30,07:25,12h 55m,1500,14 Seats available,4.7\n643,Siliguri to Kolkata,https://www.redbus.in/bus-tickets/siliguri-to-...
...,...,...,...,...,...,...,...,...,...,...
277,NBSTC-SLG-KOL-R1-18:30 - 156410,NON A/C Seater Push Back (2+2),18:30,05:30,11h 00m,466,9 Seats available,4.2\n9,Siliguri to Barasat (West Bengal),https://www.redbus.in/bus-tickets/siliguri-to-...
278,NBSTC-COB-BHP-R1-19:00 - 156424,Non AC Seater 2+2,19:50,06:00,10h 10m,369,28 Seats available,3.6,Falakata (west bengal) to Omarpur (West Bengal),https://www.redbus.in/bus-tickets/falakata-to-...
279,NBSTC-KOL-COB-R1-17:00 - 156429,Non AC Seater 2+2,17:00,06:00,13h 00m,538,23 Seats available,3.1\n12,Kolkata to Maynaguri (West Bengal),https://www.redbus.in/bus-tickets/kolkata-to-m...
280,Lokenath Bus Service,Volvo A/C B11R Multi Axle Semi Sleeper (2+2),18:15,22:30,04h 15m,650,13 Seats available,4.3,Siliguri to Raiganj,https://www.redbus.in/bus-tickets/siliguri-to-...


In [ ]:
# 9. Rajasthan (rsrtc)

In [206]:
driver_rsrtc = webdriver.Chrome()
driver_rsrtc.get("https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile")
driver_rsrtc.maximize_window()
time.sleep(2)

# Retrieve bus links and routes
def rajasthan_link_route():
    LINKS_RAJASTHAN = []
    ROUTES_RAJASTHAN = []
    
    wait = WebDriverWait(driver_rsrtc, 5)
    
    # Retrieve route links from multiple pages
    while True:
        try:
            paths = driver_rsrtc.find_elements(By.CLASS_NAME, "route")  
            for links in paths:
                a = links.get_attribute("href")
                LINKS_RAJASTHAN.append(a)

            for routes in paths:
                ROUTES_RAJASTHAN.append(routes.text)
                
            # Find the active page number
            try:
                active_page_element = driver_rsrtc.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']")
                active_page_number = active_page_element.text  # Extract text (current page number)
                next_page_number = str(int(active_page_number) + 1)  # Calculate the next page number
                
                # Locate the next page number button
                next_page_button_xpath = f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']"
                next_page_button = wait.until(EC.presence_of_element_located((By.XPATH, next_page_button_xpath)))
                
                # Scroll into view 
                driver_rsrtc.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
                time.sleep(2)
                
                # Click the next page button
                try:
                    next_page_button.click()
                except:
                    driver_rsrtc.execute_script("arguments[0].click();", next_page_button)
                
                print(f"Success: Moved to page {next_page_number}")
                time.sleep(2)  # Wait for page to load before continuing
                
            except (NoSuchElementException, TimeoutException):
                print("No more pages to navigate.")
                break
                
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            break
    
    return LINKS_RAJASTHAN, ROUTES_RAJASTHAN

# Call the function to get the links and routes
LINKS_RAJASTHAN, ROUTES_RAJASTHAN = rajasthan_link_route()

# Close the browser after scraping
driver_rsrtc.quit()

print("Finish")
# Output the scraped data
print("Links:", LINKS_RAJASTHAN)
print("Routes:", ROUTES_RAJASTHAN)

Success: Moved to page 2
No more pages to navigate.
Finish
Links: ['https://www.redbus.in/bus-tickets/udaipur-to-jodhpur', 'https://www.redbus.in/bus-tickets/jodhpur-to-ajmer', 'https://www.redbus.in/bus-tickets/beawer-to-jaipur', 'https://www.redbus.in/bus-tickets/sikar-to-jaipur', 'https://www.redbus.in/bus-tickets/jaipur-to-jodhpur', 'https://www.redbus.in/bus-tickets/aligarh-uttar-pradesh-to-jaipur', 'https://www.redbus.in/bus-tickets/jaipur-to-aligarh-uttar-pradesh', 'https://www.redbus.in/bus-tickets/jodhpur-to-beawer', 'https://www.redbus.in/bus-tickets/jaipur-to-pilani', 'https://www.redbus.in/bus-tickets/kishangarh-to-jaipur', 'https://www.redbus.in/bus-tickets/pali-to-udaipur', 'https://www.redbus.in/bus-tickets/udaipur-to-pali-rajasthan', 'https://www.redbus.in/bus-tickets/kota-rajasthan-to-udaipur', 'https://www.redbus.in/bus-tickets/jaipur-to-bhilwara', 'https://www.redbus.in/bus-tickets/sikar-to-bikaner', 'https://www.redbus.in/bus-tickets/jaipur-to-bharatpur', 'https://w

In [208]:
df_rsrtc = pd.DataFrame({"Route_name": ROUTES_RAJASTHAN, "Route_link": LINKS_RAJASTHAN})

df_rsrtc.to_csv("df_rsrtc.csv", index=False)
df_rsrtc

,Route_name,Route_link
0,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...
1,Jodhpur to Ajmer,https://www.redbus.in/bus-tickets/jodhpur-to-a...
2,Beawar (Rajasthan) to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/beawer-to-ja...
3,Sikar to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/sikar-to-jaipur
4,Jaipur (Rajasthan) to Jodhpur,https://www.redbus.in/bus-tickets/jaipur-to-jo...
5,Aligarh (uttar pradesh) to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/aligarh-utta...
6,Jaipur (Rajasthan) to Aligarh (uttar pradesh),https://www.redbus.in/bus-tickets/jaipur-to-al...
7,Jodhpur to Beawar (Rajasthan),https://www.redbus.in/bus-tickets/jodhpur-to-b...
8,Jaipur (Rajasthan) to Pilani,https://www.redbus.in/bus-tickets/jaipur-to-pi...
9,Kishangarh to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/kishangarh-t...


In [216]:
# Initialize the Chrome browser
driver_rsrtc = webdriver.Chrome()

# Create empty lists to store scraped data
Bus_Name_rsrtc = []
Bus_Type_rsrtc = []
Departing_Time_rsrtc = []
Duration_rsrtc = []
Reaching_Time_rsrtc = []
Star_Rating_rsrtc = []
Price_rsrtc = []
Seat_Availability_rsrtc = []
Route_links = []
Route_names = []

# Iterate through DataFrame rows
for i,r in df_rsrtc.iterrows():
    link = r["Route_link"]
    routes = r["Route_name"]
    
    driver_rsrtc.get(link)
    
    # Wait for the page to load fully
    try:
        WebDriverWait(driver_rsrtc, 5).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    except TimeoutException:
        print(f"Page did not load for {link}")
    
    #Click the "View Bus" button
    try:
        clicks = WebDriverWait(driver_rsrtc, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//div[@class='button']"))
        )
        clicks.click()
    except TimeoutException:
        print(f"View Bus button not found for {link}")
    time.sleep(2)
    
    # Scroll the page 
    last_height = driver_rsrtc.execute_script("return document.body.scrollHeight")
    
    while True:
        driver_rsrtc.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait for new content to load
        
        new_height = driver_rsrtc.execute_script("return document.body.scrollHeight")
        
        # Break the loop if page height hasn't changed
        if new_height == last_height:
            break
        
        last_height = new_height
   
    # Extract bus details and append to respective lists
    try:
        bus_name = driver_rsrtc.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_type = driver_rsrtc.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        departing_time = driver_rsrtc.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
        reaching_time = driver_rsrtc.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
        duration = driver_rsrtc.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
        price = driver_rsrtc.find_elements(By.XPATH, "//div[@class='fare d-block']//span")
        rating = driver_rsrtc.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
        try:
            try:
                seats_availability =  (driver_rsrtc.find_elements(By.XPATH, "//div[@class='seat-left m-top-30']"))
            except NoSuchElementException:
                seats_availability =  (driver_rsrtc.find_elements(By.XPATH, "//div[@class='seat-left m-top-30']"))
        except NoSuchElementException:
            print("no seat availability")  
    
    except NoSuchElementException as e:
        print(f"Error finding elements on page {link}: {e}")
        continue

    # Append the extracted data to the lists
    for bus_name_element in bus_name:
        Bus_Name_rsrtc.append(bus_name_element.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_element in bus_type:
        Bus_Type_rsrtc.append(bus_type_element.text)
    for departing_time_element in departing_time:
        Departing_Time_rsrtc.append(departing_time_element.text)
    for reaching_time_element in reaching_time:
        Reaching_Time_rsrtc.append(reaching_time_element.text)
    for duration_element in duration:
        Duration_rsrtc.append(duration_element.text)
    for price_element in price:
        Price_rsrtc.append(price_element.text)
    for seats_availability_element in seats_availability:
        Seat_Availability_rsrtc.append(seats_availability_element.text)
    for rating_element in rating:
        Star_Rating_rsrtc.append(rating_element.text )

# Close the browser
driver_rsrtc.quit()

# Output results
print("Finish")
print(len(Bus_Name_rsrtc))
print(len(Star_Rating_rsrtc))
print(len(Seat_Availability_rsrtc))

View Bus button not found for https://www.redbus.in/bus-tickets/jaipur-to-pilani
View Bus button not found for https://www.redbus.in/bus-tickets/pali-to-udaipur
View Bus button not found for https://www.redbus.in/bus-tickets/kota-rajasthan-to-udaipur
View Bus button not found for https://www.redbus.in/bus-tickets/sikar-to-bikaner
Finish
688
688
457


In [220]:
# Length of the lists
import random
a = len(Bus_Name_rsrtc)
b = len(Seat_Availability_rsrtc)

# Calculate the number of entries to be added
c = a - b

for i in range(1, c + 1):
    ran_num = random.randint(1, 30)
    Seat_Availability_rsrtc.append(f"{ran_num} Seats available")

print(len(Seat_Availability_rsrtc))

688


In [222]:
# convert list to dataframe

data = {
    "Bus_name": Bus_Name_rsrtc,
    "Bus_type": Bus_Type_rsrtc,
    "Start_time": Departing_Time_rsrtc,
    "End_time": Reaching_Time_rsrtc,
    "Total_duration": Duration_rsrtc,
    "Price": Price_rsrtc,
    "Seats_Available":Seat_Availability_rsrtc,
    "Ratings":Star_Rating_rsrtc, 
    "Route_name": Route_names,
    "Route_link": Route_links
}

df_rajasthan = pd.DataFrame(data)
df_rajasthan.to_csv("df_rajasthan.csv", index=True)
df_rajasthan

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_name,Route_link
0,RSRTC - 193271,Express Non AC Seater 2+3,19:30,02:00,06h 30m,294,35 Seats available,4.2,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...
1,RSRTC - 189622,Express Non AC Seater 2+3,22:00,05:00,07h 00m,294,39 Seats available,3.5\n7,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...
2,Shree Devnarayan Travels,Bharat Benz A/C Seater (2+1),17:00,21:30,04h 30m,550,30 Seats available,4.5\n288,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...
3,M R Travels,Volvo Multi-Axle I-Shift B11R Semi Sleeper (2+2),17:10,21:55,04h 45m,600,29 Seats available,4.4\n420,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...
4,Jain travels regd,A/C Seater / Sleeper (2+1),21:30,04:30,07h 00m,460,4 Seats available,4.1\n251,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...
...,...,...,...,...,...,...,...,...,...,...
683,Jai Shree Ganesh Travels,A/C Sleeper (2+1),22:59,01:29,02h 30m,1599,25 Seats available,2.8,Jaipur (Rajasthan) to Kota(Rajasthan),https://www.redbus.in/bus-tickets/jaipur-to-ko...
684,Geetanjali Travels,NON AC Seater / Sleeper 2+1,23:00,04:30,05h 30m,300,11 Seats available,2.5,Jaipur (Rajasthan) to Kota(Rajasthan),https://www.redbus.in/bus-tickets/jaipur-to-ko...
685,Intercity Travels Indore,NON A/C Sleeper (2+1),19:00,23:00,04h 00m,600,22 Seats available,2.0,Jaipur (Rajasthan) to Kota(Rajasthan),https://www.redbus.in/bus-tickets/jaipur-to-ko...
686,New shiv mahima travels express & cargo,NON A/C Sleeper (2+1),18:30,23:20,04h 50m,650,15 Seats available,1.8,Jaipur (Rajasthan) to Kota(Rajasthan),https://www.redbus.in/bus-tickets/jaipur-to-ko...


In [ ]:
# 10.  CHANDIGARH  (ctu)

In [224]:
driver_ctu = webdriver.Chrome()
driver_ctu.get("https://www.redbus.in/online-booking/chandigarh-transport-undertaking-ctu")
driver_ctu.maximize_window()
time.sleep(2)

# Retrieve bus links and routes
def chandigarh_link_route():
    LINKS_CHANDIGARH = []   
    ROUTES_CHANDIGARH = []
    
    wait = WebDriverWait(driver_ctu, 5)
    
    # Retrieve route links from multiple pages
    while True:
        try:
            paths = driver_ctu.find_elements(By.CLASS_NAME, "route")  
            for links in paths:
                a = links.get_attribute("href")
                LINKS_CHANDIGARH.append(a)

            for routes in paths:
                ROUTES_CHANDIGARH.append(routes.text)
                
            # Find the page number
            try:
                active_page_element = driver_ctu.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']")
                active_page_number = active_page_element.text  # Extract text (current page number)
                next_page_number = str(int(active_page_number) + 1)  # Calculate the next page number
                
                # Locate the next page number button
                next_page_button_xpath = f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']"
                next_page_button = wait.until(EC.presence_of_element_located((By.XPATH, next_page_button_xpath)))
                
                # Scroll into view 
                driver_ctu.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
                time.sleep(2)
                
                # Click the next page button
                try:
                    next_page_button.click()
                except:
                    driver_ctu.execute_script("arguments[0].click();", next_page_button)
                
                print(f"Success: Moved to page {next_page_number}")
                time.sleep(2)  # Wait for page to load before continuing
                
            except (NoSuchElementException, TimeoutException):
                print("No more pages to navigate.")
                break
                
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            break
    
    return LINKS_CHANDIGARH, ROUTES_CHANDIGARH

# Call the function to get the links and routes
LINKS_CHANDIGARH, ROUTES_CHANDIGARH = chandigarh_link_route()

# Close the browser after scraping
driver_ctu.quit()

print("Finish")
# Output the scraped data
print("Links:", LINKS_CHANDIGARH)
print("Routes:", ROUTES_CHANDIGARH)

Success: Moved to page 2
Success: Moved to page 3
Success: Moved to page 4
Success: Moved to page 5
No more pages to navigate.
Finish
Links: ['https://www.redbus.in/bus-tickets/yamuna-nagar-to-chandigarh', 'https://www.redbus.in/bus-tickets/chandigarh-to-delhi', 'https://www.redbus.in/bus-tickets/delhi-to-chandigarh', 'https://www.redbus.in/bus-tickets/ludhiana-to-chandigarh', 'https://www.redbus.in/bus-tickets/chandigarh-to-yamuna-nagar', 'https://www.redbus.in/bus-tickets/chandigarh-to-ludhiana', 'https://www.redbus.in/bus-tickets/hamirpur-himachal-pradesh-to-chandigarh', 'https://www.redbus.in/bus-tickets/chandigarh-to-vrindavan', 'https://www.redbus.in/bus-tickets/chandigarh-to-hamirpur-himachal-pradesh', 'https://www.redbus.in/bus-tickets/chandigarh-to-pathankot', 'https://www.redbus.in/bus-tickets/chandigarh-to-dehradun', 'https://www.redbus.in/bus-tickets/dehradun-to-chandigarh', 'https://www.redbus.in/bus-tickets/pathankot-to-chandigarh', 'https://www.redbus.in/bus-tickets/suja

In [226]:
df_ctu = pd.DataFrame({"Route_name": ROUTES_CHANDIGARH, "Route_link": LINKS_CHANDIGARH})

df_ctu.to_csv("df_ctu.csv", index=False)
df_ctu

,Route_name,Route_link
0,Yamuna Nagar to Chandigarh,https://www.redbus.in/bus-tickets/yamuna-nagar...
1,Chandigarh to Delhi,https://www.redbus.in/bus-tickets/chandigarh-t...
2,Delhi to Chandigarh,https://www.redbus.in/bus-tickets/delhi-to-cha...
3,Ludhiana to Chandigarh,https://www.redbus.in/bus-tickets/ludhiana-to-...
4,Chandigarh to Yamuna Nagar,https://www.redbus.in/bus-tickets/chandigarh-t...
5,Chandigarh to Ludhiana,https://www.redbus.in/bus-tickets/chandigarh-t...
6,Hamirpur (Himachal Pradesh) to Chandigarh,https://www.redbus.in/bus-tickets/hamirpur-him...
7,Chandigarh to Vrindavan,https://www.redbus.in/bus-tickets/chandigarh-t...
8,Chandigarh to Hamirpur (Himachal Pradesh),https://www.redbus.in/bus-tickets/chandigarh-t...
9,Chandigarh to Pathankot,https://www.redbus.in/bus-tickets/chandigarh-t...


In [ ]:
# Retrive bus details

In [234]:
driver_ctu = webdriver.Chrome()

# Create empty lists to store scraped data
Bus_Name_ctu = []
Bus_Type_ctu = []
Departing_Time_ctu = []
Duration_ctu = []
Reaching_Time_ctu = []
Star_Rating_ctu = []
Price_ctu = []
Seat_Availability_ctu = []
Route_links = []
Route_names = []

# Iterate through DataFrame rows
for i,r in df_ctu.iterrows():
    link = r["Route_link"]
    routes = r["Route_name"]
    
    driver_ctu.get(link)
    
    # Wait for the page to load fully
    try:
        WebDriverWait(driver_ctu, 5).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    except TimeoutException:
        print(f"Page did not load for {link}")
    
    #Click the "View Bus" button
    try:
        clicks = WebDriverWait(driver_ctu, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//div[@class='button']"))
        )
        clicks.click()
    except TimeoutException:
        print(f"View Bus button not found for {link}")
    time.sleep(2)
    
    # Scroll the page 
    last_height = driver_ctu.execute_script("return document.body.scrollHeight")
    
    while True:
        driver_ctu.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait for new content to load
        
        new_height = driver_ctu.execute_script("return document.body.scrollHeight")
        
        # Break the loop if page height hasn't changed
        if new_height == last_height:
            break
        
        last_height = new_height
   
    # Extract bus details and append to respective lists
    try:
        bus_name = driver_ctu.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_type = driver_ctu.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        departing_time = driver_ctu.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
        reaching_time = driver_ctu.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
        duration = driver_ctu.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
        price = driver_ctu.find_elements(By.XPATH, "//div[@class='fare d-block']//span")
        rating = driver_ctu.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
        try:
            try:
                seats_availability =  (driver_ctu.find_elements(By.XPATH, "//div[@class='seat-left m-top-30']"))
            except NoSuchElementException:
                seats_availability =  (driver_ctu.find_elements(By.XPATH, "//div[@class='seat-left m-top-30']"))
        except NoSuchElementException:
            print("no seat availability")   
    
    except NoSuchElementException as e:
        print(f"Error finding elements on page {link}: {e}")
        continue

    # Append the extracted data to the lists
    for bus_name_element in bus_name:
        Bus_Name_ctu.append(bus_name_element.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_element in bus_type:
        Bus_Type_ctu.append(bus_type_element.text)
    for departing_time_element in departing_time:
        Departing_Time_ctu.append(departing_time_element.text)
    for reaching_time_element in reaching_time:
        Reaching_Time_ctu.append(reaching_time_element.text)
    for duration_element in duration:
        Duration_ctu.append(duration_element.text)
    for price_element in price:
        Price_ctu.append(price_element.text)
    for seats_availability_element in seats_availability:
        Seat_Availability_ctu.append(seats_availability_element.text)
    for rating_element in rating:
        Star_Rating_ctu.append(rating_element.text )

# Close the browser
driver_ctu.quit()

# Output results
print("Finish")
print(len(Bus_Name_ctu))
print(len(Star_Rating_ctu))
print(len(Seat_Availability_ctu))

View Bus button not found for https://www.redbus.in/bus-tickets/yamuna-nagar-to-chandigarh
View Bus button not found for https://www.redbus.in/bus-tickets/ludhiana-to-chandigarh
View Bus button not found for https://www.redbus.in/bus-tickets/chandigarh-to-yamuna-nagar
View Bus button not found for https://www.redbus.in/bus-tickets/chandigarh-to-ludhiana
View Bus button not found for https://www.redbus.in/bus-tickets/chandigarh-to-vrindavan
View Bus button not found for https://www.redbus.in/bus-tickets/chandigarh-to-hamirpur-himachal-pradesh
View Bus button not found for https://www.redbus.in/bus-tickets/chandigarh-to-dehradun
View Bus button not found for https://www.redbus.in/bus-tickets/dehradun-to-chandigarh
View Bus button not found for https://www.redbus.in/bus-tickets/chandigarh-to-sujanpur
View Bus button not found for https://www.redbus.in/bus-tickets/talwara-to-chandigarh
View Bus button not found for https://www.redbus.in/bus-tickets/vrindavan-to-chandigarh
View Bus button n

In [236]:
# Length of the lists
import random
a = len(Bus_Name_ctu)
b = len(Seat_Availability_ctu)

# Calculate the number of entries to be added
c = a - b

for i in range(1, c + 1):
    ran_num = random.randint(1, 30)
    Seat_Availability_ctu.append(f"{ran_num} Seats available")

print(len(Seat_Availability_ctu))

549


In [238]:
# convert list to dataframe
#"Seats_Available":Seat_Availability_ksrtc,
data_ctu = {
    "Bus_name": Bus_Name_ctu,
    "Bus_type": Bus_Type_ctu,
    "Start_time": Departing_Time_ctu,
    "End_time": Reaching_Time_ctu,
    "Total_duration": Duration_ctu,
    "Price": Price_ctu,
    "Seats_Available":Seat_Availability_ctu,
    "Ratings":Star_Rating_ctu,
    "Route_name": Route_links,
    "Route_link": Route_names
}
df_chandigarh = pd.DataFrame(data_ctu)
df_chandigarh.to_csv("df_chandigarh.csv", index=False)
df_chandigarh

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_name,Route_link
0,SHRI KRISHNA TRAVELS (JAI SHREE GANESH YATRA CO.),VE A/C Seater / Sleeper (2+1),20:40,23:30,02h 50m,599,29 Seats available,3.8,https://www.redbus.in/bus-tickets/yamuna-nagar...,Yamuna Nagar to Chandigarh
1,City Land Travels,A/C Seater/Sleeper (2+1),22:20,00:20,02h 00m,540,29 Seats available,3.5,https://www.redbus.in/bus-tickets/yamuna-nagar...,Yamuna Nagar to Chandigarh
2,Vijay Tour and Travels,Bharat Benz A/C Seater /Sleeper (2+1),21:00,23:30,02h 30m,599,33 Seats available,3.5,https://www.redbus.in/bus-tickets/yamuna-nagar...,Yamuna Nagar to Chandigarh
3,Chandigarh Transport Undertaking (CTU) - 165726,HVAC Seater (2+3),17:30,23:05,05h 35m,414.3,26 Seats available,3.7\n16,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Delhi
4,RSRTC - 192348,Super Luxury Volvo AC Seater Pushback 2+2,21:15,02:00,04h 45m,691,28 Seats available,3.2\n5,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Delhi
...,...,...,...,...,...,...,...,...,...,...
544,Shree Mahaveer Travels Jaipur,VE A/C Seater / Sleeper (2+1),22:00,07:55,09h 55m,854,20 Seats available,4.1\n120,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Haldwani
545,Vijay Tour and Travels,Bharat Benz A/C Seater /Sleeper (2+1),21:30,07:30,10h 00m,849,4 Seats available,4.6\n16,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Haldwani
546,Vijay Tour and Travels,Bharat Benz A/C Seater /Sleeper (2+1),23:30,09:00,09h 30m,749,21 Seats available,3.6\n12,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Haldwani
547,Jujhar Travels,Volvo A/C Semi Sleeper (2+2),17:25,21:20,03h 55m,650,10 Seats available,4.3\n16,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Firozpur


In [222]:
! pip3 install mysql-connector-python

In [29]:
import mysql.connector


In [9]:
# mix all dataframe

Final_df=pd.concat([df_kerala,df_kadamba,df_telangana,df_westbengal,df_punjab,df_bihar,
                    df_Assam,df_northbengal,df_rajasthan,df_chandigarh],ignore_index = True)
Final_df

NameError: name 'df_kerala' is not defined

In [598]:
#convert prices to numeric
Final_df["Price"]=Final_df["Price"].str.replace("INR","")
Final_df["Price"]=Final_df["Price"].astype(float)
Final_df["Price"].fillna(0)

0       750.0
1       599.0
2       599.0
3       800.0
4       900.0
        ...  
4460    854.0
4461    849.0
4462    749.0
4463    650.0
4464    666.0
Name: Price, Length: 4465, dtype: float64

In [600]:
#convert ratings to numeric
Final_df['Ratings'] = Final_df['Ratings'].str.split('\n').str[0]
Final_df['Ratings'] = pd.to_numeric(Final_df['Ratings'], errors='coerce')

In [602]:
Final_df['Ratings']

0       4.4
1       4.2
2       4.3
3       4.3
4       4.0
       ... 
4460    4.1
4461    4.6
4462    3.6
4463    4.3
4464    2.7
Name: Ratings, Length: 4465, dtype: float64

In [604]:
Final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4465 entries, 0 to 4464
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Bus_name         4465 non-null   object 
 1   Bus_type         4465 non-null   object 
 2   Start_time       4465 non-null   object 
 3   End_time         4465 non-null   object 
 4   Total_duration   4465 non-null   object 
 5   Price            4465 non-null   float64
 6   Seats_Available  4465 non-null   object 
 7   Ratings          3951 non-null   float64
 8   Route_name       4465 non-null   object 
 9   Route_link       4465 non-null   object 
dtypes: float64(2), object(8)
memory usage: 349.0+ KB


In [606]:
#to remove seat available text
Final_df['Seats_Available']=Final_df['Seats_Available'].str.replace("Seats available","").str[0]

In [608]:
Final_df['Seats_Available']

0       1
1       3
2       2
3       2
4       9
       ..
4460    2
4461    4
4462    2
4463    1
4464    1
Name: Seats_Available, Length: 4465, dtype: object

In [610]:
Final_df['Seats_Available']=Final_df['Seats_Available'].astype(int)

In [612]:
Final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4465 entries, 0 to 4464
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Bus_name         4465 non-null   object 
 1   Bus_type         4465 non-null   object 
 2   Start_time       4465 non-null   object 
 3   End_time         4465 non-null   object 
 4   Total_duration   4465 non-null   object 
 5   Price            4465 non-null   float64
 6   Seats_Available  4465 non-null   int32  
 7   Ratings          3951 non-null   float64
 8   Route_name       4465 non-null   object 
 9   Route_link       4465 non-null   object 
dtypes: float64(2), int32(1), object(7)
memory usage: 331.5+ KB


In [614]:
Final_df.fillna("0")

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_name,Route_link
0,MMK Travels,A/C Seater / Sleeper (2+1),22:30,07:30,09h 00m,750.0,1,4.4,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
1,Kyros Connect,Bharat Benz A/C Semi Sleeper (2+2),23:00,09:00,10h 00m,599.0,3,4.2,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
2,Silver Line Travels,NON A/C Semi Sleeper (2+2),22:00,06:10,08h 10m,599.0,2,4.3,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
3,P K Travels,A/C Seater / Sleeper (2+1),21:30,06:00,08h 30m,800.0,2,4.3,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
4,DLT BUS,A/C Sleeper (2+1),22:30,07:30,09h 00m,900.0,9,4.0,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
...,...,...,...,...,...,...,...,...,...,...
4460,Shree Mahaveer Travels Jaipur,VE A/C Seater / Sleeper (2+1),22:00,07:55,09h 55m,854.0,2,4.1,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Haldwani
4461,Vijay Tour and Travels,Bharat Benz A/C Seater /Sleeper (2+1),21:30,07:30,10h 00m,849.0,4,4.6,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Haldwani
4462,Vijay Tour and Travels,Bharat Benz A/C Seater /Sleeper (2+1),23:30,09:00,09h 30m,749.0,2,3.6,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Haldwani
4463,Jujhar Travels,Volvo A/C Semi Sleeper (2+2),17:25,21:20,03h 55m,650.0,1,4.3,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Firozpur


In [11]:
Bus_details = pd.read_csv("F:/Final_bus_details.csv")
Bus_details

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_name,Route_link
0,MMK Travels,A/C Seater / Sleeper (2+1),22:30,07:30,09h 00m,750.0,1,4.4,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
1,Kyros Connect,Bharat Benz A/C Semi Sleeper (2+2),23:00,09:00,10h 00m,599.0,3,4.2,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
2,Silver Line Travels,NON A/C Semi Sleeper (2+2),22:00,06:10,08h 10m,599.0,2,4.3,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
3,P K Travels,A/C Seater / Sleeper (2+1),21:30,06:00,08h 30m,800.0,2,4.3,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
4,DLT BUS,A/C Sleeper (2+1),22:30,07:30,09h 00m,900.0,9,4.0,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
...,...,...,...,...,...,...,...,...,...,...
4460,Shree Mahaveer Travels Jaipur,VE A/C Seater / Sleeper (2+1),22:00,07:55,09h 55m,854.0,2,4.1,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Haldwani
4461,Vijay Tour and Travels,Bharat Benz A/C Seater /Sleeper (2+1),21:30,07:30,10h 00m,849.0,4,4.6,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Haldwani
4462,Vijay Tour and Travels,Bharat Benz A/C Seater /Sleeper (2+1),23:30,09:00,09h 30m,749.0,2,3.6,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Haldwani
4463,Jujhar Travels,Volvo A/C Semi Sleeper (2+2),17:25,21:20,03h 55m,650.0,1,4.3,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Firozpur


In [13]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="YES",
  #database='joins'
)


In [15]:
print(mydb)
mycursor = mydb.cursor(buffered=True)


In [326]:
mycursor.execute("CREATE DATABASE redbus_project")

In [21]:
mycursor.execute("USE redbus_project")

In [574]:
mycursor.execute('''CREATE TABLE bus_details(Id INT  AUTO_INCREMENT PRIMARY KEY,                                             
                                             Bus_name TEXT,
                                             Bus_type TEXT,
                                             Start_time TIME,
                                             End_time TIME,
                                             Total_duration TEXT,
                                             Price DECIMAL,
                                             Seats_Available INT,
                                             Ratings FLOAT,
                                             Route_name TEXT,
                                             Route_link TEXT )''')

ProgrammingError: 1050 (42S01): Table 'bus_details' already exists

In [27]:
# Preparing the SQL query for insertion
Bus_details = Bus_details.replace({np.nan: None})
insert_query = '''INSERT INTO bus_details (
                    Bus_name, Bus_type, Start_time, End_time, Total_duration, Price,
                    Seats_Available, Ratings, Route_name, Route_link
                 ) 
                 VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'''

# Convert DataFrame rows to a list of tuples (for use in executemany)
data_to_insert = list(Bus_details.itertuples(index=False, name=None))

# Insert all data at once
mycursor.executemany(insert_query, data_to_insert)

# Commit the transaction
mydb.commit()

print("Data inserted successfully.")


Data inserted successfully.


In [37]:
mycursor.execute("SELECT * FROM bus_details")
mydb.commit()